# IST 691: Deep Learning in Practice

**Homework 4**

Name:

SUID:

*Save this notebook into your Google Drive. The notebook has appropriate comments at the top of code cells to indicate whether you need to modify them or not. Answer your questions directly in the notebook. Remember to use the GPU as your runtime. Once finished, run ensure all code blocks are run, download the notebook and submit through Blackboard.*

### Setup

In [1]:
pip install transformers[torch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 8.0 MB/s eta 0:00:00:00:0100:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, BertForSequenceClassification
import torch

/Users/zanealderfer/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/zanealderfer/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
/Users/zanealderfer/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# read the downloaded dataset
df = pd.read_json('/Users/zanealderfer/Downloads/Sarcasm_Headlines_Dataset_v2 (1).json', lines = True)

In [3]:
# the training input sequence will be in variable seq_padd_train and the label in train_y
# The testing input sequence will be in variable seq_padd_test and the label in test_y
headlines = df['headline'].values.tolist()
sarcastic = df['is_sarcastic'].values.tolist()

training_size = 20000
test_size = 6709

X_train = headlines[:training_size]
X_test = headlines[training_size:]
y_train = np.array(sarcastic[:training_size])
y_test = np.array(sarcastic[training_size:])

In [ ]:
model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

In [1]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

NameError: name 'torch' is not defined

In [ ]:
train_dataset = Dataset(tokenizer(X_train,
                                  truncation = True,
                                  padding = True),
                        y_train)

test_dataset = Dataset(tokenizer(X_test,
                                 truncation = True,
                                 padding = True),
                       y_test)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    warmup_steps = 500,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
)

model = BertForSequenceClassification.from_pretrained(model_name,
                                                      num_labels = 4)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

In [ ]:
y_pred = trainer.predict(test_dataset).label_ids.tolist()
print('Confusion Matrix: \n{}'.format(confusion_matrix(y_true = y_test,
                                                       y_pred = y_pred)))

print('Precision={}\nRecall={}\nF1={}'.format(b
    precision_score(y_true = y_test, y_pred = y_pred, average = 'macro'),
    recall_score(y_true = y_test, y_pred = y_pred, average = 'macro'),
    f1_score(y_true = y_test, y_pred = y_pred, average = 'macro'),
    ))

### Q1

Seeing the performance from BERT, what do you think? Does it perform better than LSTM?

*answer here*

### Q2

The website https://cs.stanford.edu/people/karpathy/convnetjs/demo/rldemo.html has a pre-defined Deep Q-learning neural network. You can load this network by navigating to the website and pressing the button labeled "Load network from JSON".

After loading the predefined JSON, run the simulation at fast speed for 5 minutes (select "Go fast"). After that, record the "smooth-ish reward" reported. Continue to run the simulation at normal speed ("Go normal speed"), observe its behavior, then report the following:

1. What is the reported "smooth-ish" reward (running average of reward) after 5 minutes of running the simulation at fast speed?

*answer here*

2. What are you general observations about the behavior of the agent?

*answer here*

3. What are the usual mistakes it makes?

*answer here*

4. What would you propose to improve these mistakes?

*answer here*

### Q3

We have trained a substantially simpler architecture compared to the default provided by the website. The trained network has one fully connected 50-neuron layer with ReLU activation. The last cell of this notebook contains the JSON output from this network, trained for 10 minutes.

Copy and paste this JSON in the website and click "Load network from JSON" (it might take a while). After that, follow the same step above — run the simulation for 5 minutes at fast speed, etc., and answer the same questions:



1. What is the reported "smooth-ish" reward (running average of reward) after 5 minutes of running the simulation at fast speed?

*answer here*

2. What are you general observations about the behavior of the agent?

*answer here*

3. What are the usual mistakes it makes?

*answer here*

4. What would you propose to improve these mistakes?

*answer here*

---

JSON of network trained for 10 minutes. Network has one fully connected layer of size 50 with ReLU activation.:

{"layers":[{"out_depth":59,"out_sx":1,"out_sy":1,"layer_type":"input"},{"out_depth":50,"out_sx":1,"out_sy":1,"layer_type":"fc","num_inputs":59,"l1_decay_mul":0,"l2_decay_mul":1,"filters":[{"sx":1,"sy":1,"depth":59,"w":{"0":0.138693591391892,"1":0.032964652837059356,"2":-0.017167258710639087,"3":0.15372306395867213,"4":0.059793389385816444,"5":0.01515420686776161,"6":0.21363539632915277,"7":0.049451348417972565,"8":0.3497065005531873,"9":-0.07942557723180838,"10":0.23747260216902494,"11":0.018489146532115724,"12":0.32973441400986186,"13":-0.07039472738252582,"14":-0.03142532680547403,"15":-0.1102646563767508,"16":-0.005105994502608728,"17":-0.007313485896600428,"18":0.038111595103817106,"19":-0.18644728663146262,"20":-0.08868691220327608,"21":-0.08112072835408195,"22":0.19006897560360722,"23":-0.22285671956032044,"24":-0.20650844320303047,"25":-0.18976102124320113,"26":0.03490238543284647,"27":0.12151611327338423,"28":-0.04705404977654237,"29":-0.12013203305792752,"30":-0.11255476447500515,"31":-0.1762112909891835,"32":0.07461125604234645,"33":0.059486204337860824,"34":-0.19133389520034824,"35":0.03625272640628873,"36":0.007140916676585711,"37":-0.03738902313681627,"38":0.06968792950416124,"39":-0.08056469637698231,"40":0.038896833073271415,"41":-0.07018750927640056,"42":-0.07984319641686294,"43":0.12383524611490504,"44":0.10595071650424971,"45":-0.044564207584994485,"46":0.10770675457055064,"47":-0.16846750930896762,"48":0.38619620616852257,"49":-0.23771014731443266,"50":-0.04329756838603029,"51":-0.09426733259193222,"52":0.029815085598791227,"53":0.12741267200243273,"54":-0.18933139560663284,"55":-0.054339852654074955,"56":0.21161508625946054,"57":0.08509271962242367,"58":0.13428818463940315}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.02511946969085689,"1":-0.053326565786382306,"2":0.019874241787077536,"3":0.22665473285487017,"4":-0.26735958732483145,"5":-0.06879279267957394,"6":-0.06719738534137934,"7":0.026889822656457323,"8":0.034866744716666696,"9":0.0018134605954469125,"10":-0.04128519417720125,"11":0.04240481254500101,"12":-0.04338851010307286,"13":-0.16139352419852673,"14":0.1057071013782987,"15":0.10918337331898134,"16":-0.1271458808408821,"17":-0.16549634403178703,"18":0.2839140405028373,"19":-0.198108981129507,"20":0.2469272872452664,"21":-0.14937870660257527,"22":0.12758122591633875,"23":-0.24673846774205427,"24":0.10252228039650742,"25":0.1728768345936274,"26":-0.1796540205302463,"27":-0.19926375125642715,"28":-0.18925619136578303,"29":-0.11430275206632569,"30":-0.08009192943948577,"31":0.10635252470064549,"32":0.05687647725628607,"33":-0.13851106172521194,"34":-0.0459250447483706,"35":-0.012604227757087683,"36":0.09537775968184369,"37":0.009945103284323412,"38":-0.3330258233583646,"39":0.044552637413694425,"40":0.2587813042971835,"41":0.017271523898165095,"42":-0.10528893621170399,"43":-0.0074486884681848305,"44":-0.027297971272344294,"45":0.07314845167621441,"46":-0.14645045956182337,"47":-0.14446374642648652,"48":-0.09657575293235772,"49":0.0021263125645432875,"50":0.11630808762659062,"51":0.10709602002931673,"52":-0.17454207504449312,"53":0.17126680777911846,"54":0.01451740868643303,"55":-0.18367929553812642,"56":0.0800829497618227,"57":-0.059255818315291135,"58":-0.09056209646549689}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.1261202085279631,"1":0.02948857537949909,"2":0.0802122690078288,"3":0.0735912988699903,"4":-0.1587208909429649,"5":0.14502249226722944,"6":0.10466690839631418,"7":-0.003886866833902053,"8":0.23634443235332148,"9":0.33662703905622754,"10":-0.22497943431706682,"11":0.21674423328766532,"12":0.030218747849873908,"13":-0.07892717122819783,"14":0.07049225683931037,"15":0.09012506275365394,"16":0.005270005104767104,"17":0.22497657333891433,"18":-0.051895820733780254,"19":0.03549757459185266,"20":0.02312209942160409,"21":0.0725843617894554,"22":-0.023036586148650607,"23":0.06732029049005916,"24":-0.0001742510284525739,"25":-0.06872966150197181,"26":0.029943368085437662,"27":0.14201325826528916,"28":-0.01270575793896201,"29":0.044077053942244204,"30":0.10816691849984278,"31":-0.04534981502445862,"32":-0.16914109246988218,"33":0.09273804510732059,"34":0.0015263369757982678,"35":-0.08915310221186029,"36":0.16528386579978216,"37":-0.06631230465769389,"38":-0.14192089468906532,"39":-0.04260519943475755,"40":-0.22530524162716015,"41":-0.057721436487599184,"42":-0.16973132571508467,"43":0.051484406369227485,"44":-0.03180029873954009,"45":0.06405175399692778,"46":0.08307670535124599,"47":-0.014231103649414783,"48":0.12952446419691646,"49":0.07763728659019754,"50":0.02555686359686542,"51":0.24295318731369442,"52":-0.012066766141757767,"53":0.12155500450357438,"54":0.2071626110478534,"55":-0.0068918076244273395,"56":0.09486162637026951,"57":0.11159747256101392,"58":0.20253593337063125}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.42822728650577596,"1":-0.005732627379436687,"2":-0.18617190987392157,"3":-0.09415893025593153,"4":0.21522877961372452,"5":-0.1548814769606996,"6":0.009076861758203632,"7":-0.15324095376767732,"8":0.1154684403105214,"9":0.03461828994739252,"10":0.11667590722740662,"11":0.039894884053547776,"12":0.019709244170180062,"13":0.04804382156421894,"14":0.2178044294013809,"15":-0.0875053179909126,"16":-0.11916444866198798,"17":0.0060111162271986925,"18":-0.004411299964434207,"19":-0.020091758190846562,"20":-0.11989809293315727,"21":-0.04699667723848749,"22":0.32829579951987947,"23":0.2091658099467519,"24":-0.47362805336783137,"25":0.05212209484746505,"26":-0.3575577912210298,"27":0.20930914911504994,"28":0.011429508505606272,"29":0.005614992501922796,"30":0.0837387573197603,"31":0.007636202119923993,"32":-0.26314333132577256,"33":-0.10111592526280813,"34":0.04949761206854554,"35":0.12785661228355671,"36":0.023800139549327443,"37":-0.14378446922840327,"38":0.14849448505322987,"39":-0.07513300940154938,"40":-0.08069655812282264,"41":-0.10411398353578183,"42":-0.06351039323749623,"43":-0.12110198233017214,"44":-0.08789305084014128,"45":-0.10332624561348434,"46":-0.01798066628131297,"47":-0.006024321307467086,"48":-0.23777132787676675,"49":0.1382361688923313,"50":-0.2223255903752483,"51":0.12814145154474096,"52":-0.09838870838828742,"53":0.03695716769222031,"54":-0.21637811475083335,"55":0.171076998863044,"56":-0.05622172682470596,"57":-0.15791496656061607,"58":0.31989514143686903}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.0018621681218483422,"1":-0.11317473689190435,"2":-0.1203443971532811,"3":0.012305812199569733,"4":0.04084064018612088,"5":0.016154468800970618,"6":0.17426410593534725,"7":-0.13307673945965987,"8":-0.009095698764543218,"9":0.044539781021135684,"10":0.104689823031447,"11":0.03759035374636802,"12":-0.19419060385805018,"13":0.07843960641100571,"14":0.06599924953522478,"15":0.17871573038372976,"16":0.1409155087050475,"17":-0.1898628987612719,"18":0.10789030426948606,"19":0.06278377854550408,"20":0.19588917425583588,"21":0.032240838196371475,"22":-0.015250029724127787,"23":0.003056030663816259,"24":-0.07631440206219824,"25":-0.000029122190090052335,"26":-0.06941580046393529,"27":-0.06612924880041454,"28":-0.022074433835161597,"29":0.08993771612304889,"30":-0.1013849244307527,"31":0.23864108899714231,"32":-0.16663071985879402,"33":-0.07149984613037451,"34":0.14895845351000694,"35":0.056220753547314956,"36":-0.053420065856431026,"37":0.24607481742239612,"38":-0.23373544834722423,"39":-0.10308553112941726,"40":0.15119645684838184,"41":0.38139730260740606,"42":-0.09897928201459957,"43":-0.07262760096940458,"44":0.08863139993774542,"45":0.09860679671975735,"46":-0.004236691730588138,"47":0.09165633641806585,"48":0.1023745466267156,"49":-0.11984014474330747,"50":0.022266061204544135,"51":-0.004422059501789377,"52":0.13029886175344096,"53":-0.04937852620286597,"54":0.19473299683138867,"55":0.041959283161039335,"56":0.07045811436759092,"57":-0.007745940229435129,"58":-0.2180745563141153}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.011237356713193989,"1":-0.02033891372148977,"2":0.020636265454079324,"3":-0.10304750417739363,"4":0.09994772185216755,"5":-0.13349629535750615,"6":-0.03937059323535606,"7":-0.019511202197895886,"8":0.20990777259579138,"9":0.11874377680179342,"10":-0.2248090056983675,"11":0.04308584118627545,"12":-0.016143448259880535,"13":0.034053482368626785,"14":-0.09050864576939005,"15":-0.11251963035079968,"16":-0.007935259335637188,"17":-0.2676497615248683,"18":0.005123537834917778,"19":0.12536693654229064,"20":0.04403577662174918,"21":-0.15872328573310776,"22":-0.23556433244485295,"23":-0.09374964928991336,"24":0.07275022789721207,"25":-0.03352938043008022,"26":0.055077360499643645,"27":0.0512986042694126,"28":-0.056170326303100454,"29":0.011190908286305996,"30":0.14689129224318753,"31":-0.014415108990429141,"32":-0.11238866767508088,"33":-0.0700369066159496,"34":0.029914240837999047,"35":-0.07759097334749217,"36":0.12451136612026714,"37":-0.02812111678933198,"38":0.22047423724477735,"39":-0.08262690035004487,"40":0.03416794014687283,"41":-0.08927810124026316,"42":-0.20476298093433087,"43":-0.030355708362132507,"44":0.04339560682867341,"45":-0.050591534946199855,"46":-0.1506571594280724,"47":-0.04819324247281372,"48":-0.056366236106546036,"49":-0.22009655788725313,"50":-0.2887651706847012,"51":-0.07307756054249116,"52":0.14091284077430116,"53":0.1580058314242629,"54":0.12787714146229925,"55":-0.03199506847634844,"56":0.22532880235495079,"57":-0.021284202211130632,"58":-0.0639938395942052}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.041880292505162536,"1":-0.0770461121471522,"2":-0.0854600877202827,"3":0.14373021888863305,"4":-0.13244995136453755,"5":-0.00376000498743424,"6":-0.17240651332933188,"7":-0.04815093749393732,"8":0.3017881606945887,"9":-0.09610476004105105,"10":0.08918520853351711,"11":-0.13728192419976556,"12":-0.03504698166446422,"13":0.06546884856087194,"14":-0.03859090737425387,"15":0.11792021247782694,"16":0.03935703585213674,"17":-0.11305499520608546,"18":0.018559611521243694,"19":0.12668214001776215,"20":0.010925415811723009,"21":-0.09361540690636312,"22":0.22608908339936165,"23":0.07485162101084637,"24":0.3560941947618927,"25":0.33020530344104493,"26":0.2042821628364087,"27":0.08038846559478183,"28":-0.02824009466793709,"29":0.054781028700153715,"30":-0.13490679868553457,"31":0.03645848687923563,"32":-0.19177073846010145,"33":-0.008367801637786776,"34":0.07377541181739602,"35":-0.1606401709297088,"36":0.13716854121728816,"37":-0.022287150160768723,"38":0.017367725065312742,"39":-0.08736592343773776,"40":-0.15727066184288216,"41":0.03567290036172574,"42":0.017403725260186763,"43":-0.20338434735208702,"44":0.045405442189721805,"45":-0.17773703261360033,"46":-0.04964422592314735,"47":0.10958686518952285,"48":0.15375925183333333,"49":-0.0892965016036555,"50":-0.36397200981411987,"51":0.21030733642671393,"52":0.020646283471150096,"53":0.12058800602961167,"54":0.03507148678606358,"55":-0.043537551752266174,"56":0.0567350578142208,"57":-0.021452766756829566,"58":0.14749296298535003}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.2630711353297138,"1":-0.05778818662951326,"2":0.06192760316681496,"3":-0.0460878705984274,"4":-0.2316476775065201,"5":0.09373823920453775,"6":-0.20206281349210595,"7":-0.0006709095681085304,"8":0.14793831977331717,"9":-0.15317003953733857,"10":0.021502324361014485,"11":-0.14933561684450355,"12":-0.07080891769559884,"13":-0.03383227684745616,"14":-0.16319468706216375,"15":0.20326595281614437,"16":0.08648145952646888,"17":-0.008300715329954063,"18":-0.010914871180795808,"19":-0.09220424942307343,"20":0.1662359131727904,"21":-0.03187619837675895,"22":-0.05588360861573161,"23":0.1414688862643549,"24":-0.0942882483534102,"25":0.07676423867637866,"26":0.04668357673228796,"27":0.06237190077119884,"28":-0.06799495872620386,"29":0.0129487928470114,"30":0.08587132299154279,"31":-0.012007050874904139,"32":0.11783959306871372,"33":0.041632830517484516,"34":0.07581164577330624,"35":-0.006092257450335992,"36":0.12000792986505426,"37":0.02485584768170255,"38":-0.07215538994014345,"39":-0.24895596402493372,"40":0.16192743288245226,"41":-0.05563195167638995,"42":-0.09805933965112433,"43":-0.0008577455135666808,"44":-0.1713349216660495,"45":-0.21191320468122327,"46":-0.11499065111327873,"47":-0.00029686555845091783,"48":-0.371308033987455,"49":-0.031133528881293587,"50":-0.07689190411366498,"51":-0.017427282331236523,"52":-0.14048817115838672,"53":-0.12444976384754115,"54":0.025641349181603296,"55":0.09653709340306867,"56":0.06946061419065647,"57":-0.021360503429169016,"58":-0.03564445737212142}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.018570587273438053,"1":0.11333921491608413,"2":-0.11229930078503265,"3":0.2227340883357011,"4":-0.06585954466097294,"5":-0.11185810979065447,"6":0.1372734081660278,"7":-0.2405541042117691,"8":0.01308650137569257,"9":-0.07843461697377939,"10":0.0099658953182561,"11":0.14143803483193373,"12":-0.11947599548804429,"13":0.045356514171861,"14":0.1838627371943122,"15":-0.1067288146162816,"16":0.16974016540428208,"17":0.16757175252244347,"18":0.14780886060432477,"19":-0.03510380117104613,"20":0.08007922104811774,"21":0.15328449460620225,"22":-0.0012708784412894054,"23":0.20430400549445213,"24":-0.00801800377209549,"25":0.04791360730048138,"26":-0.006696602366390169,"27":-0.08550659389574829,"28":-0.07467193846697362,"29":-0.07283377253232577,"30":0.22278068024736755,"31":-0.02356874771458806,"32":0.08362441749414613,"33":0.08713894933324318,"34":0.13470039583768956,"35":0.0271121110372529,"36":-0.16525506435606768,"37":0.09249264232510707,"38":-0.17096223223991727,"39":-0.09896795474332332,"40":-0.10570499037385804,"41":0.09668069845318915,"42":0.10563232891280844,"43":-0.04372370953794313,"44":0.23826655469510852,"45":-0.06650657203451713,"46":-0.11576171030778573,"47":0.14572936110073034,"48":-0.012077414539844379,"49":0.2516083176314098,"50":-0.0423472003925885,"51":-0.17532211838002143,"52":-0.12765307408831386,"53":-0.016256962032862452,"54":0.20603872670915174,"55":-0.006300939139432159,"56":-0.003198925712271783,"57":-0.07413206872763055,"58":-0.09850783714254159}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.12064320446542758,"1":-0.10353549032657193,"2":-0.06457824917528614,"3":-0.019855982770318328,"4":-0.11055855430791317,"5":-0.22862456845850943,"6":0.05612623437973976,"7":0.07290339998376227,"8":0.11140544162335027,"9":-0.11691239898777868,"10":0.08992056315522783,"11":-0.10845930125871858,"12":-0.056702501532469446,"13":-0.024343699957570387,"14":-0.02881417205840502,"15":-0.19028038110520612,"16":-0.12471732469328914,"17":-0.043229797292426445,"18":-0.05162399472110836,"19":-0.21081554231500146,"20":0.016686431418184374,"21":0.10226738437048981,"22":-0.0044417838956486025,"23":0.050627319392397625,"24":0.14966715626066346,"25":-0.3912473454235819,"26":-0.18423808562815863,"27":0.13339706535820617,"28":0.04215465185837327,"29":0.073485520607874,"30":0.05070888679769354,"31":0.15435626492334764,"32":-0.006263932154874712,"33":-0.025272043203742392,"34":0.11854610182208035,"35":-0.00454672110331556,"36":0.0736256094424472,"37":-0.08504527278290916,"38":-0.2209912454413939,"39":-0.030622545195424046,"40":0.015097545838415957,"41":-0.011680387812684738,"42":0.025819491319774106,"43":-0.16387320306589812,"44":-0.13624601009435916,"45":0.0937930593593643,"46":-0.033178569039028084,"47":-0.17138246071008575,"48":0.06250883366237922,"49":0.10043710201716911,"50":-0.0888686120750449,"51":-0.2495052220466273,"52":0.04539994391975596,"53":-0.10714615739769472,"54":-0.08881765171365043,"55":0.10308008417856446,"56":0.13883569699438184,"57":-0.07027854115637705,"58":-0.014930437523358726}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.2593273716686441,"1":-0.033041240095499635,"2":0.14457291623314025,"3":-0.004297047885624416,"4":0.2392971135210931,"5":-0.28040073797710313,"6":0.24898376585755996,"7":0.17773181440426025,"8":0.11587944406582061,"9":0.23692373647648843,"10":0.18400100971324657,"11":-0.01847454374168399,"12":0.2504309123416468,"13":-0.07175786104050291,"14":0.022971202940236538,"15":0.33455764401395044,"16":0.10180253781516263,"17":0.05643626629298294,"18":0.32371432279895995,"19":0.03890585505554527,"20":-0.04862631539313623,"21":0.17351694766940629,"22":-0.014928271172706512,"23":0.05016293836251953,"24":0.30868999041881956,"25":-0.10087344346086137,"26":0.08435967834112068,"27":-0.10632879593078341,"28":-0.039921656618019906,"29":0.09423646769612594,"30":0.15286155369418555,"31":-0.12784715940357816,"32":0.026107038083142572,"33":-0.029581778373119,"34":-0.1786832709688837,"35":0.15900366826843276,"36":0.08754042204333883,"37":0.07225176762467746,"38":0.3063731427547294,"39":0.09161451745024887,"40":0.26121965435077116,"41":0.31075961063390634,"42":0.155924724590292,"43":0.0888945997048176,"44":0.05907499406020977,"45":-0.12663867234082418,"46":-0.26507972946898556,"47":-0.07500252182274667,"48":0.26457847760390957,"49":0.08699768871419779,"50":0.20929761167965533,"51":0.17794602545164312,"52":0.048245789593426354,"53":0.05250578757366992,"54":-0.048339097221927606,"55":-0.05498339905204174,"56":-0.06021986538603923,"57":0.08660685290437371,"58":0.08352060534470043}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.006181940980904177,"1":-0.11899995241551549,"2":0.07891018836923049,"3":-0.05789956655625898,"4":-0.30408736434998385,"5":-0.041629889397468985,"6":0.2754611905643098,"7":0.2468195524208653,"8":0.04328997723659978,"9":-0.10886141941592263,"10":-0.09019262795634223,"11":-0.014375535646246614,"12":0.06503397360594816,"13":0.05270660021299579,"14":0.045658144000758176,"15":0.20629000223878413,"16":0.02539767971974554,"17":-0.2543691622726418,"18":0.14647041397465882,"19":-0.032366205532053084,"20":0.02257159545351859,"21":-0.19722629122153085,"22":-0.002381549131593571,"23":-0.05415995631704065,"24":0.00005800918296448015,"25":0.003992423395092069,"26":0.08413161908253466,"27":-0.1243586415239545,"28":0.14826017775034694,"29":-0.08199494220976183,"30":-0.1526991858779403,"31":-0.08874127927845363,"32":-0.06466802497850394,"33":0.00775809498929149,"34":-0.09512694385007925,"35":-0.07299439067074608,"36":0.26211463082434355,"37":0.10666871292546495,"38":0.06709727023481159,"39":0.037601071554331504,"40":-0.03434347389895643,"41":-0.03841066518388101,"42":0.122031267711802,"43":-0.3575859127729534,"44":-0.008700842707039613,"45":-0.003869201990947056,"46":0.1163045303952171,"47":-0.008803580735928594,"48":0.028928618881821924,"49":-0.16410211579316492,"50":0.02076316860856109,"51":0.09658250634362452,"52":0.14056585491386459,"53":0.17040147718166426,"54":-0.023427561221950593,"55":0.040296450459136136,"56":0.19054463330485785,"57":0.12127780934963829,"58":0.06959122766777028}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.03178019701057653,"1":-0.14593548606728018,"2":0.03489908365334876,"3":-0.10534717033031318,"4":-0.06838116133061482,"5":0.011210553727515964,"6":0.003637441906433905,"7":0.002586098272629487,"8":-0.05922302053620753,"9":0.015679575378573743,"10":-0.1479118040118044,"11":0.16053367184068498,"12":0.03784166962281034,"13":0.08324465515747005,"14":-0.02028300657357857,"15":0.009749416264594522,"16":0.08986548076933851,"17":0.03761600690680999,"18":-0.1720584978704328,"19":0.12551504362163854,"20":-0.2194779752250601,"21":-0.07258759333126528,"22":0.044927372457403573,"23":-0.05736313651526601,"24":0.016581112962120524,"25":0.10676392322476867,"26":0.14778450396684847,"27":-0.08155023003156976,"28":-0.07005714783448024,"29":0.03638211393543471,"30":0.2105918188622597,"31":-0.07726970289964467,"32":0.06711152355897707,"33":0.3616430064044532,"34":0.12251866420012236,"35":0.19334356900630967,"36":-0.020126632757915677,"37":-0.22760316511396847,"38":0.08224253148473844,"39":0.060729957342681565,"40":-0.11089619811145143,"41":-0.17863309162915805,"42":-0.031608470887489595,"43":0.16481760726906816,"44":0.18251273863250506,"45":0.0774093643594849,"46":0.14422352529481092,"47":0.26864383964930433,"48":-0.3865286391738204,"49":-0.0004110358889320837,"50":0.2015615567332282,"51":-0.013063679624986738,"52":-0.1369135292296238,"53":-0.11632836361953854,"54":-0.23780687530208053,"55":0.07676244713242006,"56":-0.046986926403757234,"57":-0.0472410340627846,"58":0.18381441384584396}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.04977147933832884,"1":0.05202000196262892,"2":-0.10590677631490482,"3":0.0527663546304954,"4":-0.09702793697940221,"5":-0.07162427939928066,"6":-0.18819704885712282,"7":-0.157474597277755,"8":0.06531354866666902,"9":-0.009127970841529519,"10":-0.08753489170110412,"11":0.050226507720898664,"12":0.1274952868261886,"13":-0.09871673735491206,"14":0.12787156874746206,"15":-0.052747414630579886,"16":0.07146975282725661,"17":-0.07746414112485757,"18":0.11418402890059647,"19":0.05474294001776352,"20":-0.18510307852857988,"21":0.13958374640362728,"22":-0.05275039775845543,"23":-0.11887983320300322,"24":-0.16974415893466616,"25":-0.09435349687092033,"26":0.1407620715611049,"27":-0.010715181372689805,"28":0.036300867382688444,"29":0.00034782312020818605,"30":-0.05141797603238665,"31":0.10773541936706398,"32":-0.23892745383924055,"33":-0.2929356959963609,"34":0.08804280846522888,"35":-0.1270793728336913,"36":-0.06267599384823126,"37":0.08129112008577659,"38":0.02603290825183949,"39":-0.18966860985683914,"40":0.0776896678609291,"41":0.18939559331742312,"42":0.11283901156724445,"43":-0.26100088059547477,"44":0.052759480085234296,"45":-0.05674950175715675,"46":0.0435254235655667,"47":0.09382600605468717,"48":-0.15434174052785435,"49":0.07114099864026911,"50":0.2386077803931362,"51":-0.01856179481340013,"52":0.12227769667517434,"53":0.22117830336431618,"54":0.02458435478333582,"55":-0.06910712308519223,"56":-0.10795081128877877,"57":-0.14905737655327206,"58":0.21056733194871646}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.15448046031378662,"1":-0.015088457819410279,"2":-0.12871411199638214,"3":0.023824555303901578,"4":0.04504025535806786,"5":-0.010178249504374461,"6":0.08016267231669273,"7":-0.03061048505034291,"8":0.012766883122802624,"9":0.04765595583440604,"10":0.11745977964641023,"11":-0.2711929255625921,"12":0.2931438984025379,"13":-0.00953458046625035,"14":0.01614526336286104,"15":0.09906238225274132,"16":0.03976974418876828,"17":-0.11411884975594189,"18":0.18409941511836841,"19":-0.14268615789580777,"20":0.07650228597181491,"21":0.04189252756326725,"22":0.06539314621455251,"23":0.10432962406373919,"24":0.042126752248395044,"25":0.2505664807533035,"26":-0.1786621875551494,"27":0.15103012414855363,"28":0.0508108250315901,"29":-0.17531442135613326,"30":-0.024453023392958073,"31":0.054084909056434,"32":0.03259669264834906,"33":0.014686591137196324,"34":-0.18524255023564976,"35":-0.11357893871647856,"36":0.002268632582568612,"37":0.11152116829076743,"38":0.11486686481049803,"39":0.12839698893726317,"40":-0.025604003147769602,"41":-0.059745671767073495,"42":-0.1294763677929028,"43":-0.03538597398447962,"44":0.16038470752605033,"45":-0.23271315644432136,"46":-0.032713930301376135,"47":0.056685869066581074,"48":-0.1634056042445964,"49":0.04339028920171689,"50":-0.10860212889608226,"51":-0.0708246600488574,"52":0.11308286071260573,"53":-0.020783285980888078,"54":-0.06643296703024096,"55":0.09155073620607732,"56":-0.2802957063629347,"57":0.14348883214134547,"58":0.2821114229351907}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.2740948149570221,"1":-0.05819649291809991,"2":-0.09202328036003338,"3":-0.0742140463525409,"4":-0.03854893783172998,"5":-0.10437526914417396,"6":0.2289231694587329,"7":0.21140244312517723,"8":-0.05636764053375432,"9":0.043604484339247705,"10":0.17987563559416708,"11":-0.19294336001813284,"12":0.04491806557276087,"13":-0.14933744967340118,"14":-0.2009599343979075,"15":-0.0770996701146324,"16":-0.2775398046427317,"17":-0.10435552167500338,"18":-0.1362529270853904,"19":-0.06266442987941537,"20":-0.021065445760408755,"21":-0.251356694346729,"22":-0.11651259344154236,"23":0.1111769013321348,"24":0.18320482198101143,"25":0.14821771057049848,"26":-0.2539076057692165,"27":-0.04827535931067996,"28":-0.047551871039565784,"29":-0.19432912294675223,"30":-0.12343458720511696,"31":0.08732472907397583,"32":-0.013220924686930922,"33":0.09815178510842243,"34":-0.03358228131787532,"35":-0.09875677123470367,"36":0.032373241259224986,"37":-0.0003608710601584166,"38":0.08116570341439354,"39":0.0793844243464739,"40":0.10255872609301839,"41":0.0745413808146382,"42":-0.241350179306399,"43":-0.12256447227299204,"44":-0.2927301177526069,"45":0.10454173650400246,"46":-0.03208354285856841,"47":-0.10664567946150225,"48":0.09671585655314051,"49":-0.06386229594904454,"50":0.007092293237161879,"51":-0.14061855054055142,"52":-0.1956581437882711,"53":-0.1928094642095403,"54":-0.02183800166634686,"55":0.004922686602680511,"56":-0.14814711419697274,"57":0.059292265389075574,"58":0.1264934048988361}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.0029667742539251298,"1":0.10000086419354716,"2":0.12510035626292837,"3":-0.11911480193939722,"4":0.1478454441162177,"5":0.03651317624973025,"6":0.09174168599229451,"7":-0.1766459056460766,"8":0.08079315168928206,"9":0.1698019941218475,"10":-0.0112493970441585,"11":-0.223632155134142,"12":0.07170266899350378,"13":0.1325964620249188,"14":-0.16937763414259935,"15":0.19986506827441083,"16":-0.021887814375079938,"17":-0.3031986286197813,"18":-0.12975423102813605,"19":0.08042261680254967,"20":-0.05276753109705599,"21":-0.10426468947620164,"22":-0.13888797567633132,"23":0.2630875167723514,"24":-0.0217701140594143,"25":0.12782779104388906,"26":0.0389822275075306,"27":0.2379108993122938,"28":0.13467831006735795,"29":-0.09164422997420561,"30":-0.23142709473537468,"31":0.10949882952145099,"32":-0.06979619085974055,"33":0.29320431137974673,"34":-0.2114755055175991,"35":-0.03898144511243356,"36":0.2406015493501641,"37":-0.23321571500300992,"38":-0.03317153140771388,"39":-0.008415493521954226,"40":-0.1607542468259822,"41":-0.23799939402694664,"42":0.09806521486213285,"43":0.04228196010570423,"44":0.19549028919277406,"45":-0.20249280622634966,"46":-0.005741481066309675,"47":-0.08086309512777261,"48":-0.005818221421159732,"49":0.09675705853848833,"50":-0.1254084599951095,"51":0.012532872143874413,"52":-0.015703782289839964,"53":-0.1312944484277873,"54":-0.18824853449071427,"55":-0.01023751482779209,"56":-0.11204335153209376,"57":-0.026244448190655155,"58":-0.23919550000782505}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.235757169306775,"1":-0.07201769659728181,"2":0.02525500147007014,"3":-0.14044082078952547,"4":-0.19583168202339712,"5":-0.03422512982517145,"6":0.2518852691036267,"7":0.027242173734440964,"8":0.014367433205280911,"9":0.006016008558473821,"10":-0.19014343614882437,"11":0.11364683501467317,"12":0.015021541285517071,"13":0.17145333580101557,"14":-0.08458349932345637,"15":0.17646684223378487,"16":0.018001585886303267,"17":0.1617739298700393,"18":-0.03804233218825403,"19":0.008374509489006074,"20":0.22456258817389702,"21":0.2871085819621385,"22":0.13666233834163888,"23":0.0024185217459431916,"24":-0.23104416950746348,"25":0.1047349438180923,"26":0.14168216413997906,"27":0.05405712083161346,"28":-0.13569343454915234,"29":-0.0066622789907967855,"30":-0.29196982895470625,"31":-0.19504165551321195,"32":-0.06601218052889828,"33":0.07133840515452915,"34":-0.08178524906262098,"35":-0.15527262383190157,"36":0.024629275430007216,"37":0.06495858377938546,"38":-0.0510884136899613,"39":0.08063235444541725,"40":-0.3118699809269114,"41":0.17338495125203235,"42":0.0701349655907373,"43":-0.05698117056484258,"44":0.05286719544430358,"45":0.06876611985643702,"46":0.020509378525770727,"47":0.054672868650656986,"48":0.03489759045251599,"49":-0.07923362146877305,"50":0.07759081731266766,"51":0.23245259261048964,"52":-0.07631785074627592,"53":0.09704946723419229,"54":-0.016937956303083757,"55":-0.062273553807221764,"56":-0.06466729610884056,"57":-0.02500207075487334,"58":-0.004640220198847489}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.13048475650039754,"1":0.07426581234336457,"2":0.21827152742296918,"3":-0.20006219671957912,"4":0.1329277938809796,"5":0.10218142908591092,"6":-0.3644301494871458,"7":0.03036242857942256,"8":-0.24504212869905825,"9":0.11373908737855454,"10":-0.19696076256206707,"11":-0.016506900132371032,"12":0.3719065540233268,"13":0.028137963046184877,"14":-0.012700266410718381,"15":-0.10863990381063753,"16":-0.026610811915347674,"17":-0.25445518317556776,"18":-0.25629326795554686,"19":0.0869339711243247,"20":-0.11300662741401367,"21":0.11158276918867115,"22":-0.08367791227857767,"23":0.14598548013747367,"24":0.13009452850518674,"25":0.11323436564127706,"26":-0.2601838977280115,"27":0.10424395259074082,"28":-0.16217894344269781,"29":-0.08708474202439082,"30":0.031061197258141254,"31":-0.2506123973541459,"32":-0.01609544081304919,"33":-0.23913193794552948,"34":0.07765828859043225,"35":-0.05411500167858877,"36":0.08399837319245468,"37":-0.18544069173793104,"38":0.1306451484706877,"39":0.13924485120892988,"40":-0.14559623958740453,"41":0.030043757252696868,"42":-0.017879848873748296,"43":0.1857557903450522,"44":-0.014522529837790947,"45":-0.05627346990701699,"46":0.017013909220411398,"47":-0.09455165307029281,"48":0.001656521478368498,"49":0.12759294954698053,"50":-0.06711773443690436,"51":-0.16790580753024148,"52":0.17409528246686032,"53":0.14911112873846463,"54":-0.1621072208210794,"55":0.17043824153336265,"56":0.0001973760667517264,"57":0.04520531888846552,"58":0.1412410986488691}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.1523832554553589,"1":0.14028776899551773,"2":-0.005095262023193761,"3":0.03832294867654896,"4":0.06407692873642176,"5":-0.092493023578489,"6":-0.019217402196389954,"7":0.12358789671777703,"8":-0.09201375676107236,"9":0.24528322016314388,"10":-0.08860741586474204,"11":0.15355886735927807,"12":0.15664277380438202,"13":0.18431756192660248,"14":0.06245819876363355,"15":0.3496210590129854,"16":-0.20072199119772471,"17":0.2301560698598086,"18":0.3130567889442202,"19":0.1131688034059797,"20":-0.17404112959526566,"21":0.32762428465538346,"22":0.003270319689622323,"23":0.04686072664186576,"24":0.07665470318939324,"25":0.05113871278035125,"26":0.006666175639621789,"27":0.2129413602737618,"28":0.02748394232855893,"29":-0.08999054577257029,"30":-0.0666013825086351,"31":-0.05971603516041422,"32":0.026725430254628264,"33":0.21103572306226007,"34":0.07639172680789394,"35":-0.023990619405632154,"36":-0.19288841666125206,"37":0.07318687736060595,"38":-0.006362939608288018,"39":-0.05554321138712639,"40":0.055471319365978135,"41":0.04288363457710345,"42":0.21878141937588233,"43":0.11219667307617993,"44":-0.07810538525093177,"45":0.14474369355992522,"46":-0.020946555270335378,"47":0.07938722848759247,"48":0.09535759862166852,"49":-0.12733686404122346,"50":0.00042116030179982614,"51":-0.2842451872320209,"52":0.06950920322003284,"53":0.08092313399665553,"54":0.12554248432777063,"55":-0.18251762794682352,"56":0.09609616724992008,"57":0.07936883167465328,"58":-0.024969134917974603}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.08475316724138443,"1":0.0806804915888653,"2":0.07196242439055349,"3":0.0781333979624092,"4":0.010489474803232396,"5":-0.041489624602778885,"6":-0.1394489900982988,"7":0.2697773778536967,"8":0.0599906880527974,"9":-0.05421355610003925,"10":-0.0748510084391466,"11":0.04436364442448906,"12":-0.11586606615528185,"13":0.3790836213932616,"14":0.13713111725775462,"15":0.037115285085331,"16":0.14331182472915044,"17":-0.03054265324685204,"18":0.188436532448254,"19":-0.03799264304952566,"20":0.08401917864046302,"21":-0.2635180510486768,"22":-0.49376661651790776,"23":-0.14994134778557303,"24":0.03896023794224056,"25":-0.16091074137945482,"26":0.0008602057217963058,"27":0.17409932298518735,"28":0.3496860853037574,"29":0.02830869852583483,"30":0.03526809713353491,"31":-0.06919783280583632,"32":-0.1340514494448143,"33":-0.05118991023161045,"34":0.04467542417334719,"35":0.3641823915634155,"36":-0.04335552985761066,"37":0.28178459017159424,"38":-0.1136960838807081,"39":-0.20356549881785466,"40":0.062230081549951105,"41":0.08916430161196555,"42":-0.0948822732481965,"43":-0.12217591418943467,"44":-0.04520477325900446,"45":-0.11105852820594177,"46":-0.013181306237851442,"47":-0.09708576478857647,"48":0.01832832205805142,"49":-0.07522982495213998,"50":-0.026767698599363322,"51":0.14799751361295993,"52":0.06895856116969684,"53":0.22450245287437642,"54":0.061269645706061575,"55":-0.18640889370163452,"56":0.12637233207300558,"57":0.17747716650209788,"58":-0.1940305881093319}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.06801857437963237,"1":-0.12896717035557498,"2":-0.1318991733715812,"3":-0.14292117084791608,"4":-0.08091726719003597,"5":-0.08748120494450913,"6":0.04410221962469538,"7":-0.021948238477965346,"8":0.1427073594475653,"9":0.03417571607943642,"10":0.03528956934560983,"11":0.09996141785021108,"12":0.08202716559855902,"13":0.25670781309450763,"14":-0.06075126977220946,"15":0.20658361725753416,"16":-0.024657047770823,"17":0.05836177533471184,"18":0.2706544906861812,"19":-0.00039337662042604037,"20":-0.09294633191610109,"21":0.12160915732141146,"22":0.03816196465756414,"23":-0.06381503509025255,"24":-0.16462855031608106,"25":0.024400296490067,"26":-0.015197444457681905,"27":0.09103377686480543,"28":0.06254162799438359,"29":-0.02149824594160249,"30":-0.06596077986951303,"31":0.006484874644720563,"32":-0.07049698717894648,"33":-0.03914710980766492,"34":0.10504987397466516,"35":0.02572118007368646,"36":-0.029820039000071078,"37":-0.14610436430411952,"38":0.24735131582004952,"39":-0.07844663447322876,"40":0.17866764603586743,"41":0.24041024497160038,"42":-0.17891828233041976,"43":-0.23075715348878315,"44":-0.15014551737319462,"45":-0.3699237346217667,"46":0.04396609685035521,"47":-0.010002856489261339,"48":-0.10651397600982404,"49":0.2719078408568724,"50":0.08980376403678891,"51":-0.02082090694884152,"52":0.09508470444976617,"53":-0.20305559642592025,"54":-0.08711401338889332,"55":0.009232698872869817,"56":0.03152376544666966,"57":0.03473053718107178,"58":0.017334188798923653}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.1792441508831851,"1":0.07418100799300069,"2":-0.03012228634209918,"3":-0.1425592685884779,"4":0.06440586805217292,"5":-0.1614450536743686,"6":0.2396617870699384,"7":-0.032252412871230436,"8":0.14717229777840285,"9":0.09868873849386933,"10":-0.24387249616454995,"11":-0.003370597075670095,"12":0.18386465501689622,"13":-0.12425223735781736,"14":0.08179974966837582,"15":0.0862245286057639,"16":-0.05403449629306506,"17":-0.22056093300060273,"18":0.03921641646960865,"19":-0.07240051293641116,"20":0.013328836815894645,"21":-0.16451139535311193,"22":0.01987494524841111,"23":-0.3661766365668803,"24":0.05045839228178073,"25":-0.11394345384996049,"26":0.03505370077316254,"27":0.01786989645659689,"28":-0.011897846961126621,"29":-0.0254875679354935,"30":0.11229999851198338,"31":0.17395202790780218,"32":-0.07974262376113112,"33":-0.17166934691751062,"34":-0.09090979781112998,"35":-0.28107775356847575,"36":-0.009024772569837431,"37":-0.26047679910000743,"38":0.08008140688199848,"39":0.018077796361787433,"40":-0.01706480990410391,"41":0.10821689534691864,"42":0.10902875734311293,"43":-0.1291070318392385,"44":0.059349593400669955,"45":0.30487499450328454,"46":-0.06902232960525856,"47":0.04194153722929014,"48":0.0741181744583949,"49":-0.01208934304278749,"50":-0.026136927981556,"51":-0.07578508076573258,"52":-0.01766731377835495,"53":-0.1271909175821788,"54":0.030807357043695312,"55":0.010343348689592155,"56":0.0019684615023818864,"57":-0.03506338348237468,"58":0.08150820554914108}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.02049414155919677,"1":-0.13160494059005864,"2":0.02663020272466667,"3":-0.028914555682755628,"4":-0.20286256463384095,"5":-0.017508766281675133,"6":0.07082030059631106,"7":0.029774180772200107,"8":-0.17921435727302965,"9":0.04281923487851291,"10":-0.03263399721940684,"11":-0.21524608608716136,"12":-0.05379290181367806,"13":0.16901371369398668,"14":-0.0936011620623904,"15":-0.03871064300775019,"16":0.12579751242937598,"17":-0.016607721731089396,"18":-0.2756949881545084,"19":-0.04885671018697092,"20":-0.11306240654565462,"21":-0.09344485372322628,"22":-0.07142600749561484,"23":0.07466355122254423,"24":-0.17549285694133204,"25":0.03803156026039466,"26":-0.04260669196884691,"27":-0.1452855710900225,"28":0.23616483653709025,"29":-0.11192458358013445,"30":-0.08698385133319737,"31":0.17672223214378915,"32":-0.030982391820223307,"33":-0.19667738802519696,"34":0.06471268398684063,"35":-0.05986721569498413,"36":-0.17433333599763662,"37":0.025188002255524523,"38":-0.2998324018269849,"39":0.14887092322203216,"40":0.2725325129999737,"41":0.1019463937418917,"42":-0.07228709550063217,"43":-0.15395766842971667,"44":0.2405542652186437,"45":-0.07253546522625358,"46":0.28157287881348436,"47":-0.03734905725972959,"48":-0.25537837779093864,"49":-0.1769086388616599,"50":0.023248602365285347,"51":-0.27621954008201843,"52":-0.049115210985601926,"53":0.01255416399540519,"54":0.2032994211468068,"55":0.11087967856131921,"56":0.09423778976071462,"57":-0.02043617480047302,"58":-0.07662618877006298}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.10817554662735952,"1":0.26303382831575356,"2":0.11680328626325051,"3":0.25242547844237684,"4":0.05106138966085952,"5":0.19200231812106794,"6":0.2659598946859015,"7":0.004295717150376351,"8":0.03497245188061635,"9":-0.08782976968315623,"10":-0.20236201826925468,"11":0.09475169020951607,"12":-0.07914556312985113,"13":0.19417700647416447,"14":0.12496530086829051,"15":-0.0032988794006802965,"16":0.039993083157367014,"17":0.07077413962108413,"18":0.22950612767905892,"19":0.31037990104775115,"20":0.06638016993601167,"21":-0.24538550515661794,"22":0.2177949136836417,"23":0.026450880722732734,"24":-0.02574894229540689,"25":-0.04191554781426467,"26":-0.16949216133892586,"27":-0.04348051953276522,"28":0.13684001301043142,"29":-0.11055158451450321,"30":0.12930381277276384,"31":-0.05153822992559334,"32":0.23615242682681384,"33":-0.2175692159962697,"34":-0.08029673779570094,"35":0.11644140219376718,"36":0.16125447021317169,"37":0.03723710746289358,"38":-0.013586870465872845,"39":-0.056505484591290095,"40":-0.15887045410663625,"41":-0.07436809151915624,"42":-0.06847699962668544,"43":0.13509843401747137,"44":-0.07450379237685667,"45":0.11274708947264815,"46":0.07904930322194029,"47":-0.05597224512089032,"48":-0.18351466291215304,"49":-0.035028252223586215,"50":0.0011957709906048792,"51":-0.2798873912687125,"52":0.04130378037548644,"53":-0.11294594661684472,"54":0.14507693758970394,"55":0.15445948969389836,"56":0.041486256479933106,"57":-0.03984593709647767,"58":0.10252450159353461}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.05759011951720312,"1":0.04347111306817745,"2":-0.13217494136924698,"3":0.12038156540554693,"4":0.23478250154239755,"5":-0.0790798982203846,"6":-0.0518697959783763,"7":-0.04242020310008545,"8":-0.04703618742567914,"9":-0.0776558619529572,"10":0.18084725341673327,"11":0.024260194297644793,"12":-0.14307876149820076,"13":0.20809597682330666,"14":0.1908107852925388,"15":-0.10401600427059958,"16":-0.04228098030099469,"17":0.14377220741011423,"18":0.009568728227316201,"19":0.23420712528625368,"20":0.05918247660210599,"21":0.021315015676749927,"22":0.12099558539912546,"23":-0.11214863409004196,"24":-0.0855179574486524,"25":0.03018486548664579,"26":-0.03441245775178045,"27":0.08179224171568823,"28":0.005937212094146084,"29":-0.013271874134360418,"30":-0.06377294457210847,"31":-0.07502163033006506,"32":-0.2287948330233131,"33":0.26395819547638916,"34":-0.08978891012374048,"35":-0.060132078900022215,"36":-0.018064953801810682,"37":0.012806990637363826,"38":0.01048650308054635,"39":-0.005345468023062042,"40":-0.002362285473545161,"41":0.14430716291045323,"42":0.09749600883459397,"43":-0.09573748529457662,"44":0.008753263637759887,"45":-0.106762931139799,"46":0.0488000508464851,"47":0.08295311346140138,"48":0.1087876363323466,"49":-0.17223468381048077,"50":0.11759523284893197,"51":0.07827478645832066,"52":-0.043134494856918434,"53":0.02016929806151425,"54":-0.17260475768152653,"55":-0.18294297488587224,"56":0.19097581895993707,"57":-0.2875399907425568,"58":0.04209914731815416}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.012796475436391532,"1":-0.19401268971025773,"2":0.0556570590757241,"3":0.20098953636550082,"4":-0.243379414558778,"5":-0.07514868258282131,"6":0.06676231699467487,"7":0.056923765192052805,"8":0.024341178615802504,"9":0.257452629903464,"10":-0.004614470924487311,"11":-0.13072036160278613,"12":-0.024757357373784223,"13":-0.03153658584115251,"14":0.12370963243947164,"15":-0.046339552863829445,"16":-0.1964636879943274,"17":-0.026125788901715474,"18":0.07035688096455148,"19":-0.32045693775924405,"20":-0.1781977036500535,"21":-0.14615027350221454,"22":-0.20387119614496993,"23":0.041695523844728306,"24":0.055873213715874726,"25":0.1665475939225734,"26":0.039387662515766864,"27":0.15351584317589587,"28":0.23490771696405238,"29":-0.09562216657019101,"30":-0.1482651284895002,"31":0.012120796487669892,"32":0.15301321514851232,"33":-0.11549372505692163,"34":-0.06422694489014309,"35":-0.04577117988831315,"36":0.005292130100851412,"37":-0.1432689662499587,"38":0.0360202295680005,"39":0.13098721137091426,"40":-0.00028638431503178857,"41":-0.1414756523539069,"42":-0.02101919043134662,"43":-0.03136446718454375,"44":0.059638514953547044,"45":-0.09507054245955943,"46":-0.16208102151879955,"47":0.08667873714069259,"48":-0.1236593693607944,"49":0.06088432647509138,"50":-0.1304962606645029,"51":-0.15487569132462387,"52":-0.12005118051598179,"53":0.011710449293324583,"54":-0.03109378016624213,"55":0.22338882905817709,"56":0.09045924728577431,"57":0.18565094399326318,"58":0.20884121765137112}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.0017544517981839109,"1":-0.0954582153029394,"2":0.30052925152312565,"3":-0.04227616856139694,"4":-0.014988233729548684,"5":-0.16711454683612342,"6":-0.0834745230380588,"7":-0.1346540010349213,"8":0.03923174057376057,"9":0.2755558290055648,"10":-0.10150688798414989,"11":-0.2843591439592562,"12":0.13485576768858212,"13":0.268884960921372,"14":0.05185644623640134,"15":0.099636139703783,"16":0.15534788546343703,"17":-0.09611585442828594,"18":0.15030459553883593,"19":0.04168151838876651,"20":-0.18196488817930292,"21":0.24383754805530522,"22":0.18206016316023957,"23":-0.04994346885598059,"24":0.044723240318063996,"25":0.02274330228607685,"26":0.06076193095917817,"27":-0.03463690130073774,"28":-0.18314389093225497,"29":0.011252892362950542,"30":0.06140078674645888,"31":-0.010026295869392755,"32":0.12398081205746152,"33":0.08718649898331333,"34":0.0027915091755786894,"35":0.0036052281005898262,"36":0.104968309482762,"37":-0.005078899737778958,"38":-0.24940677451333687,"39":0.0828959209920576,"40":-0.23011036931850548,"41":-0.11829937674706004,"42":0.037764504921112114,"43":-0.11110755455834129,"44":-0.1386550448623731,"45":0.09678181163953258,"46":-0.026001563549426315,"47":0.0052050552542602015,"48":-0.19335440407702356,"49":-0.153719308534879,"50":-0.11051207827246301,"51":0.09818180351159804,"52":-0.05574098773601794,"53":0.14863481463802247,"54":-0.03308355479427092,"55":0.12437838562780854,"56":0.09465978270076851,"57":-0.029743816332460798,"58":-0.02584807626990786}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.008691438462169212,"1":0.16279880375796363,"2":-0.05399947541601204,"3":-0.02320815448294326,"4":-0.036688904264654314,"5":0.14998603525596993,"6":0.01249050858543457,"7":-0.020726656672581976,"8":-0.0827404133967314,"9":-0.03138983750429085,"10":0.11219818394551213,"11":-0.06187418796707084,"12":-0.0397805909522126,"13":-0.011950140233198969,"14":-0.06985004778223468,"15":-0.21350187189149236,"16":0.06879034317571128,"17":-0.028783253561014823,"18":0.05763930389361291,"19":-0.05222732847678751,"20":-0.08079038486683893,"21":0.17888739572535742,"22":-0.06136714213648445,"23":-0.003466054155396123,"24":-0.12317472313548034,"25":-0.11839267185544475,"26":0.1503420857257364,"27":0.024716087084704814,"28":0.05108730198359283,"29":0.07197169039992317,"30":-0.1941162164743562,"31":-0.042301945235917765,"32":-0.19233627887106555,"33":-0.04491922251912049,"34":-0.27473026220352753,"35":0.13973615205223328,"36":-0.3016235188152804,"37":0.011205235141941485,"38":0.020174261200747143,"39":-0.017181224619965664,"40":0.054608563532814174,"41":0.011232362491474003,"42":-0.0020262763846384253,"43":0.08791788662675983,"44":-0.10826502353268622,"45":-0.00874360620706808,"46":-0.022636641954958542,"47":0.021817788545590983,"48":0.018290098257855728,"49":-0.1039467546571437,"50":-0.17231507878988944,"51":0.13751302911109095,"52":-0.27519299036703493,"53":-0.08646608022833008,"54":0.06363510355656951,"55":-0.08406219837801732,"56":-0.01674894122529013,"57":-0.016809088200323553,"58":0.21138539889902402}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.23504073647581758,"1":0.19696890644786777,"2":-0.1409988916206594,"3":-0.0461703103055926,"4":-0.2379571135274718,"5":0.08723459056557019,"6":0.03975729467221453,"7":0.018447346360039345,"8":-0.1077987088491785,"9":0.03653515253260492,"10":0.13517033109005697,"11":-0.12586718035259534,"12":-0.051485309666260595,"13":0.03328340337467044,"14":0.02515724454381822,"15":0.105689279299248,"16":0.14438711056334633,"17":-0.008175690588120616,"18":0.06326779996129657,"19":0.22629780487070067,"20":0.12827431605072717,"21":-0.05872849437180639,"22":0.033925039080474687,"23":-0.10297541429920688,"24":0.18776244078688675,"25":0.09565371904908343,"26":-0.01564822496181695,"27":0.040397606765991155,"28":0.037329899006993975,"29":0.2200768527937715,"30":-0.04672119936132019,"31":0.09880501445945392,"32":-0.16853989810066833,"33":-0.16692324072249887,"34":0.059609994622475494,"35":-0.02468853080652331,"36":0.02477691873896138,"37":0.04429902803623107,"38":-0.2354883617003029,"39":-0.05949883575957538,"40":0.08144290576165242,"41":0.06978776753098831,"42":-0.1732171606318307,"43":0.027911071130778547,"44":0.056885713822645324,"45":-0.11470340618456554,"46":0.09990886471522066,"47":0.21849721962430513,"48":0.1577748439687332,"49":0.011592524355662754,"50":0.15100951307753682,"51":-0.011050129584190096,"52":-0.032277816338673465,"53":0.11316483551977957,"54":0.10911443156576632,"55":0.09900768227021747,"56":-0.07832496716126995,"57":-0.09482315445734894,"58":0.11033392208456251}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.011089001973360615,"1":0.09745127665421474,"2":0.05350975302517408,"3":0.009724440132202071,"4":0.18272497055946013,"5":0.027252019214397325,"6":-0.15390040000247782,"7":0.07632994314759116,"8":-0.04926199421293939,"9":0.13435310998052294,"10":0.11704971523707754,"11":-0.008802292870583683,"12":0.04698703941373629,"13":-0.10430500213129962,"14":-0.06170869383748903,"15":0.13600282899522084,"16":-0.16964984682627332,"17":-0.007105074560043443,"18":0.17418635226382792,"19":-0.048274212569987034,"20":-0.04975223483444331,"21":0.1749566308741288,"22":0.206564314289956,"23":-0.044199002784991025,"24":0.040404101945880275,"25":-0.25837384917520484,"26":0.08224440043914186,"27":-0.06630374973000427,"28":-0.33100987729332854,"29":0.09920056474374032,"30":0.04205408447345582,"31":-0.04839003052453418,"32":0.2051872082728687,"33":-0.07294915345391974,"34":-0.24529111726602956,"35":0.1250139546426976,"36":-0.11530061982020934,"37":0.040585943534410314,"38":-0.22288392134605628,"39":0.008987542440456714,"40":0.1582440531254515,"41":0.08000147706871745,"42":-0.30052614129851213,"43":0.09491010564730391,"44":0.2164153414921445,"45":-0.02633232029447109,"46":-0.19841829258946747,"47":0.05159987941436686,"48":-0.0030405759377006253,"49":-0.11753216722501157,"50":0.27120128011482736,"51":0.0024676614940001102,"52":-0.2556713523219922,"53":0.11056489314285567,"54":-0.044834904939392996,"55":0.13432029346828375,"56":-0.02578870277812761,"57":-0.19945658939847147,"58":0.009579442959428627}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.03138398277680246,"1":-0.09221036255298792,"2":0.03086999155922449,"3":0.20887639435735053,"4":0.09331729739020775,"5":0.022856965685148882,"6":-0.02952176188995271,"7":0.21397168130331337,"8":-0.06083931980180838,"9":0.08911584787474523,"10":0.03815457396482581,"11":-0.08596276147352667,"12":-0.14521944511198592,"13":0.15445511504443352,"14":-0.012533877059040157,"15":-0.05014151067680443,"16":0.13101625969532935,"17":0.0637693485220385,"18":0.22118570699988102,"19":0.07543208068177837,"20":0.0001928984372535505,"21":0.13202068541658118,"22":-0.2029370247496777,"23":0.18404106427525707,"24":-0.19650785288801328,"25":0.05750483044967733,"26":-0.07982269917060796,"27":-0.03580079248999534,"28":-0.0043753785048620255,"29":0.04840736994098495,"30":0.0952944154596879,"31":0.0649813367016291,"32":0.15297838921615867,"33":-0.09377429457846853,"34":0.2518043446286689,"35":0.19763882000842717,"36":-0.05641020890335656,"37":-0.0636052356271217,"38":-0.007754612536141276,"39":-0.01678645329048723,"40":0.06473472688913595,"41":0.015646931934982396,"42":-0.10759937089500572,"43":0.03568942372252858,"44":0.15279251878481404,"45":0.022057298888886048,"46":-0.17639458704856678,"47":-0.05491146007432871,"48":-0.19086884292669162,"49":0.09007017215910111,"50":0.24619980352300824,"51":0.0035204731510134588,"52":0.010601204556509933,"53":0.0753179741284883,"54":0.05467992869769597,"55":0.19968297978451757,"56":-0.19195102645400727,"57":0.07044730213531163,"58":0.055967603120735225}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.010992752936906216,"1":-0.1792370025547988,"2":0.19233102861254897,"3":-0.11701162494215199,"4":0.019599070314700553,"5":-0.08660563885909803,"6":0.10108576175123936,"7":-0.11035360852230164,"8":-0.1783991077446149,"9":0.11957914094163317,"10":-0.09988478179000651,"11":0.026529758573776308,"12":-0.20497747183060408,"13":-0.09588344270206427,"14":0.16228578282693498,"15":0.05143579595680223,"16":-0.04255828670292409,"17":0.1006600772063517,"18":-0.25306554554366417,"19":-0.033664930576348076,"20":0.09995854682946378,"21":0.0985863424685075,"22":-0.1063769689307172,"23":-0.01585952153626915,"24":0.00957105942734762,"25":-0.13830373758342426,"26":0.09438549653938728,"27":-0.20615124252132766,"28":0.16442315879018832,"29":-0.04384338184193089,"30":-0.19392381332682923,"31":-0.015751186706173616,"32":-0.36835063331465273,"33":0.05974357640889912,"34":-0.10192080960714855,"35":-0.08748656464839698,"36":-0.044881934321005894,"37":-0.004393534158963351,"38":-0.02712176482989337,"39":0.04814210658901396,"40":-0.07444195722095405,"41":-0.08812566158618108,"42":-0.00782110910115159,"43":0.20139340972186234,"44":-0.02532165790447361,"45":0.3227881750174223,"46":0.059991945981971925,"47":-0.08165726250768322,"48":0.2377236121349802,"49":-0.016056059815834793,"50":-0.04380107840858482,"51":-0.0390508242087356,"52":0.028519534389395827,"53":0.1689022156872321,"54":-0.025105910216578934,"55":-0.027251814418282727,"56":0.05362958059143186,"57":-0.06966981473412925,"58":-0.02132726520101023}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.0819166973487674,"1":0.2386959290637117,"2":0.13679662881564447,"3":0.1931462555567991,"4":-0.2250948607807794,"5":0.18335389801116445,"6":-0.08023782396577706,"7":-0.05229146872024109,"8":0.008022035537531657,"9":0.03193748769346632,"10":-0.19681882411155416,"11":-0.01295864472874773,"12":0.23267718641793017,"13":0.06025785970962418,"14":-0.157758437329248,"15":-0.0913939978491304,"16":0.2252499704560349,"17":-0.09237042027342356,"18":-0.05669031142437485,"19":0.020204869755306158,"20":0.11290345000634033,"21":-0.1972955964711347,"22":0.158829969838334,"23":-0.07002794817088027,"24":-0.03487001505787198,"25":0.1855020436655605,"26":0.15594273037393042,"27":0.06372981510412531,"28":0.07782226051015158,"29":0.1513928573544695,"30":0.12165993644628167,"31":-0.08319897050390633,"32":0.014572255726354507,"33":0.3215592158313439,"34":0.1671171680922237,"35":0.11690277763397225,"36":0.24391599103063602,"37":-0.09177943446785984,"38":0.10287506478518019,"39":-0.07505873612354484,"40":-0.18308582745971558,"41":0.020629362985896318,"42":-0.06852050253295207,"43":0.02773356492752294,"44":0.14037621234447253,"45":-0.056134842369345336,"46":0.031537772396875295,"47":-0.15939483455262915,"48":-0.09642452448175366,"49":0.10256408697941516,"50":-0.049065613232805144,"51":-0.043324444216522276,"52":-0.03355963045331219,"53":0.15841622824117826,"54":0.04671634300893993,"55":0.15113111678902186,"56":0.014698859931603319,"57":0.07394017939172451,"58":-0.05444976822742658}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.10750994491490763,"1":-0.09486021270514093,"2":-0.05227639397213913,"3":-0.18916584217707194,"4":-0.014496199041489194,"5":0.17532078614681282,"6":-0.030475841200726425,"7":0.046108150130819606,"8":-0.2203300116094014,"9":-0.08897105299869937,"10":0.12393723888168387,"11":-0.08549375600079574,"12":-0.26285933145710616,"13":-0.1661412363041944,"14":0.1302011633477444,"15":-0.03837606321207463,"16":-0.09527776800491852,"17":0.0023248893646546135,"18":0.1172159586764407,"19":-0.10252861057772468,"20":-0.0009778717326197066,"21":-0.16184117069833887,"22":0.0830621218077344,"23":0.022877894080960644,"24":0.36796598557629534,"25":-0.2980412362065294,"26":-0.01937744820767805,"27":0.15519416206628273,"28":-0.01802232692263491,"29":0.23966689325687063,"30":0.02302160967055641,"31":0.009507862916350081,"32":0.09737124004866768,"33":0.09289700752319352,"34":0.3225170558632383,"35":-0.013754026000169438,"36":-0.12542742039274687,"37":0.013459511501587397,"38":0.003144028696658741,"39":-0.07047379499359706,"40":0.005868731406829039,"41":0.1431031631044955,"42":-0.10512056758942649,"43":-0.261691898993598,"44":-0.03967373561802418,"45":-0.10665285762839292,"46":0.07095483422953913,"47":-0.09309358401900195,"48":-0.11345423513197228,"49":0.001777512946173206,"50":-0.02042976626916585,"51":-0.03232855005166098,"52":0.07819573353726894,"53":-0.01676440004710737,"54":-0.07934898332013006,"55":-0.21446149146720697,"56":-0.3537293881190752,"57":-0.10236254743563669,"58":-0.0417423487609531}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.04939970459408387,"1":0.2082720764233236,"2":0.16655242364385442,"3":0.12437122056080346,"4":0.12412121878843847,"5":-0.28768244233181206,"6":0.29329584943503134,"7":0.16970135278664242,"8":-0.29017626455193873,"9":-0.07057501449442015,"10":0.07614206396441205,"11":0.04126425064400177,"12":0.19146448758648948,"13":-0.033726353071339435,"14":-0.1093723932764371,"15":-0.11960840056985586,"16":0.14302121515886945,"17":0.07434619978558804,"18":-0.07676795626484993,"19":0.20195395895644785,"20":0.12000281492752592,"21":-0.10580578155051143,"22":0.04096815416421239,"23":-0.0675241891997947,"24":-0.07636800473935952,"25":-0.001967135648685332,"26":0.1389920981118443,"27":0.02820963202804545,"28":0.08291398804262551,"29":0.049712381680151886,"30":0.030090459546182575,"31":-0.037666090493836,"32":-0.10529533175986998,"33":0.0058037874735085,"34":-0.036940488149434535,"35":0.049147140000212135,"36":-0.026421139667609946,"37":-0.10427683005907518,"38":-0.0006793622014375826,"39":-0.15309268155545822,"40":-0.09740484829956934,"41":0.03370139862628397,"42":-0.07271332392463588,"43":0.261436290614472,"44":-0.09673414675771186,"45":0.12788462436359355,"46":-0.12402344870969732,"47":0.04133896642996927,"48":0.036831920588054165,"49":0.12409672880716341,"50":0.02336151719477058,"51":0.2321688615631933,"52":-0.10484950405934305,"53":0.1261672462649183,"54":-0.07280518079425707,"55":0.07938951592190696,"56":-0.11637297338902784,"57":0.007966591443911958,"58":-0.06013128819463753}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.043651694384483075,"1":-0.11100833658857182,"2":-0.02071382855105539,"3":0.16900692068385645,"4":-0.14651559488760948,"5":0.054009565273323026,"6":-0.022084955462127014,"7":-0.03088445423076989,"8":0.062167440150154664,"9":-0.033004432701843923,"10":0.12582109055990479,"11":-0.038037030599585644,"12":-0.04885361276988035,"13":0.0060088640854952914,"14":-0.0455468283124389,"15":0.13102306348479853,"16":-0.16839022404587983,"17":0.038329966615334415,"18":0.29674374589276237,"19":-0.01821839034662979,"20":0.01699671263311087,"21":0.18218602179750507,"22":-0.025717082296196354,"23":-0.19842403644184808,"24":-0.001219148768077334,"25":0.07118719107854442,"26":-0.01707393934130628,"27":0.12518058875241014,"28":0.18860280445795335,"29":-0.01967819587570356,"30":-0.16671583880323798,"31":-0.06910817340980144,"32":0.06924513538657606,"33":0.020095726849055723,"34":0.07646599465097406,"35":-0.11245722988493843,"36":0.0348025057314479,"37":0.08934287831586206,"38":-0.09321695607419668,"39":0.0067104400067596786,"40":-0.1729890093043076,"41":0.08659665576472311,"42":0.16131469381686642,"43":0.19501446503153314,"44":0.1398890938440123,"45":-0.07940646855265333,"46":0.11817084171400223,"47":-0.061387388659323985,"48":-0.0972891859452908,"49":-0.08707564728826084,"50":-0.13371991635074693,"51":-0.020790015722206134,"52":-0.10051453370862352,"53":-0.017394507536005948,"54":-0.031127926558214582,"55":-0.2318789768377481,"56":-0.01484481512363386,"57":-0.020971884602060138,"58":-0.034037669746605666}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.2545164412263865,"1":-0.013836926973984911,"2":-0.04995890895619939,"3":0.08529601551896318,"4":-0.24392264980578074,"5":0.13237267917591397,"6":0.14014998651702243,"7":0.08454363446400877,"8":-0.012853480908977494,"9":0.11253440762608323,"10":-0.09307777374719091,"11":0.15878566747979295,"12":0.06975300273091861,"13":-0.006150468636994715,"14":0.05636014307934422,"15":0.2016898246584496,"16":0.17579156872857848,"17":0.19249391117403136,"18":-0.03352473889015068,"19":0.26145476849440147,"20":0.24580039784495072,"21":0.0526175266698545,"22":-0.007769572881227166,"23":0.012049924111587748,"24":0.05400406333674515,"25":-0.2718475801276909,"26":-0.11381886339115821,"27":0.15865598349547377,"28":-0.047248855667936004,"29":-0.08352526083624437,"30":0.1278803042526787,"31":0.1071950056754356,"32":0.03160225718752498,"33":0.21317477331553533,"34":-0.045516978864781704,"35":-0.21534660346282963,"36":0.08845829286381958,"37":0.03316620270261126,"38":-0.07510528351581577,"39":0.047090825570484114,"40":-0.010801025160073812,"41":-0.01655751873929969,"42":-0.04161837079114111,"43":-0.20639506742244468,"44":0.038143286498958776,"45":0.15675925319265288,"46":0.05438739980578966,"47":0.025024583699130683,"48":0.11761908198438854,"49":0.13282065775980378,"50":0.013539560024974417,"51":0.15509934361353608,"52":-0.08188263560533546,"53":0.10269175430201129,"54":0.08258966817498731,"55":0.0974296798510591,"56":0.0716068497641894,"57":-0.013361201031833114,"58":-0.011613392124200629}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.09688502361194029,"1":-0.034959172945669754,"2":0.2762675933044887,"3":0.07119834173629401,"4":-0.04892349946682224,"5":0.022136902607533,"6":-0.050595240157569,"7":0.03338786999110735,"8":0.135485619972434,"9":-0.05521563241761483,"10":0.03990067334655628,"11":-0.24505545371981563,"12":-0.1937512040596121,"13":-0.0977878498798271,"14":0.06669084700937826,"15":-0.013711268841837007,"16":0.04417057458254846,"17":-0.12510136600919192,"18":-0.09298092469390754,"19":-0.005099531215416073,"20":-0.03839654708444415,"21":0.23941722697951534,"22":0.14202093344041036,"23":-0.040717552633626315,"24":0.03977423575897251,"25":-0.123120671777461,"26":-0.14129279874832332,"27":0.1127701325913099,"28":0.00922733571714916,"29":0.0007821723860605745,"30":0.11355084573934694,"31":-0.09715341791361365,"32":0.08374063277284201,"33":-0.10498006921344075,"34":-0.015495875347387212,"35":0.15756150876424121,"36":0.11090390698420086,"37":-0.14412394579113316,"38":0.11115690002397816,"39":0.011293889884805413,"40":-0.0836264309646718,"41":-0.06698332796980207,"42":0.038833751230092334,"43":0.1226896593018995,"44":-0.12207841452176282,"45":0.11267041722030265,"46":-0.1539477220359785,"47":0.05193469243693355,"48":-0.09081141803950121,"49":0.08042224738434864,"50":-0.01922912791892186,"51":0.06678580933742968,"52":0.1630644080571098,"53":0.21944240520102237,"54":0.003755004496237037,"55":-0.02005732698353322,"56":0.06464938874051349,"57":-0.032448596402113,"58":-0.030736511433533526}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.0637100815301988,"1":0.025863879323016415,"2":-0.08076085579955189,"3":-0.047091817421437254,"4":0.23951764107246754,"5":-0.2414021451680891,"6":-0.07628626145616635,"7":-0.18607675694750406,"8":-0.11301959167755105,"9":0.0006184584634431002,"10":-0.04180031832460473,"11":0.04649431135106523,"12":0.14222522693785103,"13":-0.16219936972714644,"14":0.026620603851587033,"15":0.06952986991554463,"16":-0.07785725083885668,"17":-0.09578713098798539,"18":-0.19531366015857238,"19":0.022554015656224016,"20":0.028208854638149303,"21":0.15809564867712653,"22":-0.2688211680485731,"23":-0.030544370414952875,"24":0.11269735793626798,"25":-0.017810069178387726,"26":0.00139186804966912,"27":0.12352293706474059,"28":0.0011976747439885744,"29":0.03786413384158369,"30":-0.15687958179080322,"31":0.12416596935937946,"32":0.13040502004183438,"33":-0.016200866918446008,"34":0.1315086146901591,"35":-0.04529465549050494,"36":-0.13394633662942104,"37":-0.005542214364839862,"38":-0.05690916264085846,"39":0.24256203337104268,"40":-0.09439654888147313,"41":-0.06452871734193474,"42":0.07111283077711765,"43":-0.24374692312744414,"44":-0.04290733404409805,"45":-0.08710744788882659,"46":0.15360218118125066,"47":-0.00622404420407035,"48":0.06540917459012534,"49":0.04381905423363507,"50":-0.07764634791499014,"51":-0.012295616670456137,"52":-0.19000518161126315,"53":-0.09726176983767036,"54":-0.07109512226799572,"55":-0.03549801039759403,"56":0.07441201710864308,"57":0.0679394286197867,"58":0.004471014599013903}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.1950893817913183,"1":0.19803620781805906,"2":0.23477627871672307,"3":-0.11148616707850492,"4":-0.024745570585887784,"5":-0.01943711130360241,"6":-0.010640151218460259,"7":0.191731384416919,"8":0.20751843218087346,"9":-0.17793055474733632,"10":0.0231212333604081,"11":0.09795755079843392,"12":-0.006868796873605856,"13":0.09622085355406536,"14":0.021419621991382785,"15":-0.012108035876901844,"16":0.0836359599719417,"17":-0.039793892467380854,"18":-0.08478580828759345,"19":-0.22847402731994743,"20":-0.10832206467080689,"21":-0.11047350891124139,"22":-0.05145195723889588,"23":-0.24220277070078797,"24":-0.15769348614617934,"25":-0.009962231357969689,"26":-0.04973763094871658,"27":-0.12059939415385455,"28":0.28527373113582377,"29":-0.0059906635630053015,"30":0.0018541586435060332,"31":-0.09793986897922466,"32":0.2398333437687767,"33":0.016589916797347868,"34":-0.12243416211981979,"35":-0.0871785174101431,"36":-0.03062415169458842,"37":-0.06563560720536338,"38":-0.07895958848418194,"39":0.1715720859542155,"40":0.07936551369286984,"41":-0.0012071180844846227,"42":-0.09692863728839124,"43":0.10536770761874958,"44":0.14316704264729904,"45":-0.1589788492166979,"46":-0.06017519385367851,"47":-0.09479682291294451,"48":-0.053151602282972193,"49":0.06883031761558088,"50":-0.02308029950931088,"51":-0.06447629203124795,"52":0.1335934214714072,"53":0.2479090087948522,"54":0.04139391999367639,"55":-0.16793705758075494,"56":-0.013858943282178555,"57":0.1635323047663902,"58":-0.11183758250369749}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.26697447330697266,"1":-0.18232970966088216,"2":-0.051508720902788206,"3":0.034121829818757665,"4":0.086436037692555,"5":0.025652127535022752,"6":-0.15108716303104797,"7":0.009924151022439012,"8":-0.0038524044158862597,"9":0.10183053999755151,"10":0.07208293920553595,"11":-0.187590186933211,"12":0.03280343358743459,"13":0.021701200697804412,"14":0.14666759824973652,"15":0.1872614321384045,"16":0.014302282627292197,"17":0.024630414241512633,"18":-0.04309490426839084,"19":-0.007344810201144502,"20":0.011703752690409732,"21":0.1546794972272604,"22":0.1355402827476154,"23":-0.1694916432061495,"24":0.04419605439938131,"25":-0.06383424006363325,"26":-0.031580067141868874,"27":0.20473308090557107,"28":-0.1865468474934827,"29":0.07234983286656554,"30":-0.10398664029582352,"31":0.025438947162876646,"32":0.08263746643995594,"33":0.037877236581052826,"34":-0.19902990912972965,"35":-0.08395256190536986,"36":0.03112083288937308,"37":-0.019541767139082948,"38":-0.10943183750605409,"39":0.021298643356594704,"40":-0.14410449927255503,"41":-0.09809535814183583,"42":0.3088750547884405,"43":-0.054775111188795196,"44":-0.2991709782409363,"45":-0.30322417818088954,"46":-0.014110188042849263,"47":0.12405597015052183,"48":-0.27541024972208195,"49":-0.24903243800737926,"50":0.013246489878610596,"51":0.1850408951984872,"52":0.0455449108463648,"53":-0.022479584775183834,"54":-0.04689834380923598,"55":0.2708060691574159,"56":-0.11322452808749249,"57":-0.019529546985318603,"58":-0.09484827198160355}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.19368901233694918,"1":0.09720026286640791,"2":0.03852479232213697,"3":-0.14317341079951895,"4":-0.13923155573974955,"5":-0.0366174068996778,"6":0.053799578770199336,"7":-0.06288025122844734,"8":-0.10427880447547777,"9":-0.0033978646524014844,"10":0.039770255800317546,"11":0.18006555088603832,"12":0.11385111489965978,"13":-0.0377184004577753,"14":-0.22179060404647102,"15":-0.018538257813703322,"16":0.2095646479493085,"17":0.13551356855843238,"18":0.24856940750699458,"19":-0.028617127964735874,"20":-0.03274790182255034,"21":0.13436862620987644,"22":-0.15153890068936884,"23":0.025439226159448613,"24":-0.03791229745485213,"25":0.005683469199903975,"26":0.055759489031811604,"27":0.10986804399616219,"28":-0.12373324074260596,"29":0.015817712542239123,"30":-0.06081019872782804,"31":-0.17622716486075304,"32":-0.25412290341835947,"33":-0.024698363179583467,"34":-0.0284441617043706,"35":0.15736435293877074,"36":0.11524378085366481,"37":0.026440404833958844,"38":-0.0365814377891235,"39":-0.16090339920015842,"40":0.1024180534819454,"41":-0.030216985571777936,"42":-0.19669689983995428,"43":0.11596341403097617,"44":0.079224948251526,"45":0.05490935477525072,"46":-0.0009957109491008208,"47":0.24958171061247614,"48":0.01898814810913367,"49":0.20868965601538172,"50":-0.0006171893173825796,"51":0.15079730062832175,"52":-0.01235972420729976,"53":-0.30253808674915805,"54":0.12184233425248704,"55":-0.034261470709905,"56":-0.13018518834600906,"57":-0.10432707194190796,"58":0.14220601459927007}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.02548944982885605,"1":0.0035101274067361205,"2":-0.016635910880750213,"3":0.06422432024430784,"4":0.043583606749986924,"5":0.22435049977777535,"6":-0.09316977826040326,"7":-0.06189885452415949,"8":0.2613932064813311,"9":-0.2092180901934373,"10":-0.022205151777469376,"11":-0.16676710915343435,"12":0.024436090237689986,"13":-0.06559056565361353,"14":0.1604925729792725,"15":0.09762829722270053,"16":-0.07213362816859038,"17":0.16186170692957239,"18":0.05470450088996226,"19":-0.17038190428847544,"20":0.0562201810078719,"21":0.12580762781262123,"22":-0.0769491943627057,"23":0.031308890625448206,"24":-0.17077951327560917,"25":-0.00867375285601941,"26":-0.11733410111113536,"27":-0.05676375837095837,"28":-0.09292755513901763,"29":0.18629298502374272,"30":0.1262952578909132,"31":-0.16404943864141316,"32":-0.020565315732332327,"33":0.08957433971293864,"34":-0.4006389158954225,"35":-0.17728707288982393,"36":-0.004022665274012487,"37":0.13091280714553385,"38":-0.0230593386395881,"39":0.06553943061077615,"40":0.06054459169841476,"41":0.08248347860075833,"42":-0.0949891156998431,"43":-0.07124705594466177,"44":-0.07713966043545178,"45":-0.21774071248092836,"46":-0.09426879133777902,"47":-0.06740885051794893,"48":-0.05793252790834025,"49":0.005851761061106862,"50":0.25310747567650405,"51":0.21845385458685043,"52":-0.2129396847998104,"53":-0.27551682616198286,"54":-0.05858654601385175,"55":-0.10125440915992669,"56":-0.14563249862870714,"57":0.021314585434908717,"58":0.08140170700516088}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.08034372728931055,"1":-0.1100173977278747,"2":-0.24647546471579734,"3":0.15779005092612555,"4":-0.1360665097595936,"5":-0.3270626025525608,"6":0.048870763632295026,"7":0.10766356937924787,"8":-0.1792157641547751,"9":-0.07837766029375616,"10":-0.07586986226158822,"11":0.12299500081513925,"12":0.009008096513157943,"13":-0.1346251682782452,"14":-0.13727862342987734,"15":0.23616421199158733,"16":-0.015967848739169416,"17":-0.023669821142943383,"18":-0.16841158718541618,"19":-0.05451712795033664,"20":-0.1781356140593165,"21":-0.1347848552577259,"22":-0.030614084397622723,"23":-0.18616722759057036,"24":-0.08927179391621912,"25":0.19233792827894242,"26":-0.04088252472559318,"27":0.018959026539339893,"28":-0.13642708179934349,"29":-0.11553631775438981,"30":-0.14627941912024317,"31":0.13006743119751363,"32":-0.19053102310600847,"33":-0.09806097619533592,"34":0.015240784399019972,"35":-0.10810490425772958,"36":-0.04907453988782271,"37":0.06607595857156992,"38":0.03237886912025928,"39":0.2518836099096785,"40":0.0019295505970552814,"41":-0.14130872319659318,"42":0.1308268572806031,"43":0.010483514198638721,"44":0.2412310638934034,"45":0.1824107467506934,"46":0.175048783880666,"47":0.05322645449943236,"48":-0.190142239415641,"49":-0.09804947183072442,"50":-0.0004051391239958262,"51":-0.09954721524353172,"52":0.14719359458925954,"53":0.03982952656152111,"54":0.2590812860093141,"55":-0.04158999343509813,"56":-0.260307676981287,"57":-0.044643689539430036,"58":0.021936284323288348}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.12119065967230287,"1":-0.058098243887035286,"2":-0.06839492631661494,"3":0.270508288640868,"4":0.009951836603058184,"5":0.08103788718265396,"6":0.23009204806690386,"7":0.13744758442067606,"8":0.07912522883394514,"9":0.20360942365654483,"10":-0.3387543085737926,"11":0.03406881775572453,"12":0.09951364226786501,"13":0.06190638875877424,"14":0.15122963779587345,"15":0.16180793556107242,"16":-0.18441919931884937,"17":0.11385941148351789,"18":0.2812177833244172,"19":0.11930972393547756,"20":0.08457366429959158,"21":0.17356881080246367,"22":-0.07358445718048506,"23":-0.015524127544596346,"24":0.16336928648580104,"25":-0.10306397814772707,"26":0.16478446683116169,"27":0.26227223939533045,"28":-0.09741995358844703,"29":-0.023641752597524785,"30":-0.033185923711612116,"31":-0.05527473583553539,"32":0.10674714497630752,"33":-0.2310620588735593,"34":-0.1262334571055855,"35":0.144686718592817,"36":0.24131500547783205,"37":0.14609783532675963,"38":0.21002384392624052,"39":-0.035532993877026096,"40":0.07582180935382488,"41":0.17895327056532043,"42":0.11169260802009179,"43":-0.14102141654670544,"44":-0.04303117815161521,"45":0.05679000442608376,"46":0.009284111954573602,"47":0.0953406140374326,"48":0.20022556217958842,"49":-0.10431706115565344,"50":0.02389855797349788,"51":0.22702560903355568,"52":0.09850176255174135,"53":0.0373165699037025,"54":-0.02849867738892843,"55":0.24501843001530554,"56":-0.14489980518252343,"57":-0.030784490956764998,"58":0.026984049751148185}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.13572059670922942,"1":-0.01670097524493955,"2":0.0346261083685525,"3":0.03736792401236313,"4":-0.06456590484394746,"5":-0.07860892076756905,"6":-0.21126730452881123,"7":-0.14841296141619534,"8":0.15251479353793135,"9":-0.06743665854000219,"10":-0.19300982706174988,"11":0.16412521746368586,"12":0.04224549084483645,"13":0.14300821985577905,"14":0.1431504778861611,"15":-0.026035106408175426,"16":-0.0591782375652693,"17":-0.03502454974754969,"18":-0.04952347377225425,"19":-0.09364851500491543,"20":0.11417217231025005,"21":0.03298473192193057,"22":0.026883128690974405,"23":-0.08944021164380157,"24":0.011714267574749561,"25":-0.02131479169286587,"26":0.16069430349235256,"27":0.3042392543839619,"28":-0.002745672593000419,"29":0.07236890800763962,"30":-0.11104894774538497,"31":-0.11953764070188183,"32":0.1867020733359056,"33":0.12607225831943597,"34":-0.0066608810596109685,"35":-0.043025469568029634,"36":0.14447089583155573,"37":-0.16404760191883802,"38":0.029039242529308684,"39":0.11406236617446076,"40":-0.06542071881492822,"41":-0.06496141824981999,"42":0.06915141300096715,"43":-0.09110233069276283,"44":-0.06490127439412575,"45":-0.05990217886600451,"46":-0.19426760045496388,"47":-0.1819828224627339,"48":-0.13309311628572051,"49":0.18011777011878183,"50":-0.08940918068170713,"51":0.058963654252262965,"52":0.0623762893712096,"53":0.2516804810788369,"54":0.12359532658148852,"55":-0.044080514228588795,"56":-0.18832897080935793,"57":-0.1268004116427414,"58":0.06972544656434293}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.06482497365309822,"1":0.009636018492466937,"2":0.014311417350533779,"3":-0.019699777632867614,"4":0.09582764627773144,"5":-0.12535473640156738,"6":0.1279165412829173,"7":-0.19013182828083594,"8":0.08864023911074433,"9":0.049222366968482145,"10":0.022641440175327338,"11":-0.11455334912276625,"12":-0.08088928880581675,"13":-0.10210927312936538,"14":0.005981353900932688,"15":0.06609427029032183,"16":-0.04091728253116859,"17":0.15885185204287852,"18":-0.03996325499661545,"19":0.030999185785478174,"20":0.055365018430688445,"21":0.06479915185203827,"22":0.21451234052988497,"23":-0.02269302750429774,"24":0.19070637723203074,"25":-0.20505413665115899,"26":0.14773715156390713,"27":-0.07232017609570789,"28":-0.06308902596285972,"29":0.014981583432954861,"30":-0.110200380266058,"31":-0.060098712971390564,"32":0.0367946280073712,"33":0.17596712987548765,"34":-0.2611372902995965,"35":0.09217462545709866,"36":0.10836674551746603,"37":-0.08740353525328706,"38":-0.09335684117750284,"39":-0.018763632840707964,"40":0.061093386809192324,"41":0.01232041673583383,"42":-0.134673316397654,"43":0.23346171826406506,"44":-0.014852590690621488,"45":-0.06121461614417683,"46":0.16464725196688676,"47":0.01664917234823876,"48":-0.04202597146752326,"49":-0.19902899163890894,"50":-0.001541953681475945,"51":-0.09075123572292101,"52":0.10614730005797533,"53":-0.13909816590152083,"54":0.12975766977050007,"55":0.20272630690976462,"56":-0.09506311333410347,"57":-0.08115481912496739,"58":-0.02546946723092381}},{"sx":1,"sy":1,"depth":59,"w":{"0":0.23733534311453963,"1":0.29729542109062895,"2":0.07764206674667369,"3":0.08276732268916193,"4":-0.052286411970387414,"5":-0.10795155291346285,"6":0.16670235202221387,"7":0.02812751124029991,"8":-0.014300713397158863,"9":-0.001864421509907343,"10":-0.02581738237387141,"11":0.1724103256309084,"12":-0.06093416049566009,"13":0.047663382059028134,"14":-0.015229016592155984,"15":0.27435964813977576,"16":0.06554949834101097,"17":0.04868696636574586,"18":-0.28496539790817477,"19":0.09466593924485724,"20":0.22409512939849074,"21":0.06683177327829722,"22":0.012444316407304345,"23":-0.006076157586675679,"24":-0.07210521855839505,"25":0.1328634469557259,"26":-0.16775316766256107,"27":0.15174755190126535,"28":-0.15593388335262132,"29":-0.0641776497067161,"30":0.06851520743304093,"31":0.1725077303885357,"32":0.025312062111421472,"33":-0.13973524112001798,"34":-0.0036093851522139434,"35":-0.2252395616346686,"36":0.08919059312595141,"37":0.2330161558497135,"38":-0.033716198569740126,"39":0.12144086555485994,"40":-0.1961291242574513,"41":0.04387202210185547,"42":-0.1497665171011462,"43":-0.14136796168388277,"44":0.04603192905045261,"45":0.27253241921045585,"46":0.2689395670839386,"47":-0.007103563098661048,"48":-0.01606857953146869,"49":0.0837975403446662,"50":-0.031265784458531405,"51":-0.11581200639646219,"52":0.0182020935891716,"53":-0.1910589432369641,"54":-0.25603201531033604,"55":-0.02760024740980097,"56":-0.06599400627693845,"57":-0.023704908247182365,"58":-0.08296338754303019}},{"sx":1,"sy":1,"depth":59,"w":{"0":-0.06746713386796191,"1":0.03938718853007381,"2":0.05636542070909951,"3":0.3861428920500588,"4":0.11899081721920861,"5":-0.0687643777306129,"6":-0.022780423224978346,"7":-0.02570791898030592,"8":-0.16118604481790022,"9":0.07550930754682922,"10":-0.12247601573069462,"11":0.031527437385192966,"12":0.03881862125412411,"13":-0.19556572965304056,"14":-0.06160389520044007,"15":0.011056574189839356,"16":-0.05230571718527516,"17":-0.12714635000347557,"18":-0.14425355153888406,"19":0.037900500940107586,"20":-0.28754368925444446,"21":-0.1500301086781574,"22":-0.08206068242864771,"23":-0.05511728787571864,"24":0.12743323347807678,"25":0.03267507263872844,"26":0.13705569607669882,"27":-0.07388313127083787,"28":0.04439019178828228,"29":0.0029243742696073707,"30":0.27118148513785717,"31":0.28181529702919783,"32":-0.0313086381779217,"33":0.029544261639831218,"34":0.2004281261329637,"35":0.07244919175598517,"36":0.01940422140687912,"37":0.1668448944899592,"38":0.045959255710151845,"39":0.10025635002940579,"40":0.12822638661619365,"41":-0.02276735787824662,"42":-0.047485278781075856,"43":0.06714781578023757,"44":-0.12661279398070258,"45":0.058943616484385564,"46":0.25218406122358733,"47":0.01095408932035909,"48":-0.03904030570146562,"49":0.06050105143211131,"50":-0.17389565011666827,"51":0.05546984714561439,"52":0.1898946553911135,"53":0.08699528273355164,"54":0.1474352905604828,"55":0.028985404991926502,"56":-0.10476901763056859,"57":0.11430738285111575,"58":0.1390247162194861}}],"biases":{"sx":1,"sy":1,"depth":50,"w":{"0":0.09608637346229586,"1":0.1012450746844254,"2":0.09513546912617986,"3":0.09982313178009612,"4":0.1040871240163283,"5":0.10243432791812736,"6":0.0929393217914958,"7":0.10145605564946396,"8":0.10337650599580757,"9":0.0985930453781185,"10":0.10195489439097571,"11":0.09655620001359033,"12":0.09797211156576571,"13":0.10066993796002824,"14":0.09883897597910118,"15":0.09904524454875682,"16":0.0996205645019896,"17":0.10078256959458726,"18":0.10024849022118244,"19":0.09829395566986822,"20":0.10047549292574816,"21":0.10199025990611398,"22":0.09808972449317246,"23":0.09863380811308368,"24":0.10150365784033083,"25":0.09553562943176364,"26":0.09719268012030953,"27":0.10108261292157444,"28":0.10281769220485502,"29":0.09858896530938371,"30":0.09610799730728926,"31":0.09898421367366529,"32":0.10015266312673395,"33":0.09858713673591962,"34":0.1,"35":0.09826766482097977,"36":0.09800762707515981,"37":0.0970603553659744,"38":0.09650403813245198,"39":0.09800663984587488,"40":0.10001988364818172,"41":0.10040311901977678,"42":0.09758690560957767,"43":0.09790327810276511,"44":0.09752580864007857,"45":0.09902026148607125,"46":0.10079653496444879,"47":0.100640653848361,"48":0.09927804097036694,"49":0.09834401383500233}}},{"out_depth":50,"out_sx":1,"out_sy":1,"layer_type":"relu"},{"out_depth":5,"out_sx":1,"out_sy":1,"layer_type":"fc","num_inputs":50,"l1_decay_mul":0,"l2_decay_mul":1,"filters":[{"sx":1,"sy":1,"depth":50,"w":{"0":-0.06376383057708272,"1":0.08480196194271107,"2":0.08885709428961534,"3":0.02453940157856707,"4":-0.1602449390554649,"5":0.12268384732002485,"6":0.145504873918892,"7":0.004343159327604786,"8":0.03873930520423088,"9":0.18389250480995292,"10":0.5208695590481821,"11":-0.08496507000895805,"12":-0.19118243013883418,"13":-0.00177739961222776,"14":-0.043598100577777084,"15":-0.10465019827846808,"16":-0.011426209107278729,"17":0.0842995530765278,"18":-0.22342430418812395,"19":0.2590665615856398,"20":-0.1259815311996369,"21":-0.08300494581871652,"22":0.0916224238230947,"23":-0.2018276280158954,"24":-0.03451777231222069,"25":0.061208901328180854,"26":-0.1688726730886934,"27":0.1769027306557125,"28":0.2981949623785678,"29":-0.12251823448637351,"30":0.049055129027740944,"31":0.09500025311908288,"32":-0.057651493678570934,"33":0.15450924626833162,"34":0.0641981359814876,"35":0.19048008727903454,"36":0.04472141026756401,"37":0.20514039672379278,"38":-0.02307246905966064,"39":-0.006879040740186083,"40":-0.06880232187917157,"41":0.041874888063989944,"42":0.016125796058813955,"43":-0.11828263599714091,"44":0.04009505363411455,"45":0.1916298230529421,"46":0.1361651823984231,"47":-0.0053629167958626515,"48":0.05493100867406169,"49":-0.10841343255808393}},{"sx":1,"sy":1,"depth":50,"w":{"0":0.05322462908146454,"1":-0.03398955507163837,"2":0.17764940799968063,"3":-0.058130070020055144,"4":0.18084116714846718,"5":0.157010055353041,"6":-0.13361323955344273,"7":-0.05475899341608988,"8":0.09255278899947074,"9":-0.12699636925869415,"10":0.20233997687148356,"11":0.20089867774243686,"12":-0.09160793722852456,"13":0.02313104167797932,"14":0.001374137468417532,"15":-0.004136930840092728,"16":-0.2194133357115861,"17":-0.1347466618100763,"18":0.1296867040296108,"19":0.2697513812473125,"20":-0.04856220027911512,"21":-0.008276881551719852,"22":-0.04115376156246468,"23":-0.3591356427025405,"24":-0.12391325195149942,"25":-0.12217137382853951,"26":0.007230360232250512,"27":0.07693336840295457,"28":0.020245987539647643,"29":0.14895401303785671,"30":0.22749880264242708,"31":-0.09042728158004094,"32":-0.14849441381151146,"33":-0.11600745987750433,"34":-0.1442129133575573,"35":0.08734737557003519,"36":-0.07046603642062109,"37":0.18086622973813937,"38":-0.011530408811208877,"39":-0.04571892665595721,"40":-0.1800091767319424,"41":-0.008260184734594023,"42":-0.05725988458703927,"43":-0.13622641127199256,"44":-0.09061925511187956,"45":0.3056342234378542,"46":0.15589736538162738,"47":-0.16495682937048614,"48":0.05810592182178069,"49":0.025497584035137393}},{"sx":1,"sy":1,"depth":50,"w":{"0":0.0644584170136789,"1":0.21850376788475018,"2":-0.003109922260460515,"3":-0.1706168563498583,"4":0.07719817797792398,"5":-0.17089141771072403,"6":0.32095165080817384,"7":-0.16464265258399266,"8":0.16942178926307397,"9":-0.06949465415451236,"10":0.24417204422346755,"11":-0.01681448908320104,"12":-0.03719609748514636,"13":0.005581634692578574,"14":0.2096276144030227,"15":-0.1345497194314701,"16":0.1763686053162996,"17":0.31236821233182593,"18":-0.05531083676566693,"19":0.2291154873028486,"20":0.015569542237036825,"21":0.04649141801173027,"22":-0.04433972031623884,"23":-0.2285258577535532,"24":0.20176129105205282,"25":-0.011764140606010907,"26":0.026119357892915956,"27":0.13570812526197182,"28":-0.06404556512610816,"29":-0.21917129538098018,"30":-0.31827671155824305,"31":-0.17175854652966577,"32":-0.24531170118451417,"33":0.09530561939620873,"34":-0.2763455691634526,"35":0.01336464512517907,"36":-0.3203836192986145,"37":0.2178421193148887,"38":-0.1800388013556309,"39":0.11586428909713328,"40":-0.22163477168042878,"41":0.19318682252236877,"42":-0.07635386801732119,"43":0.13893232474277217,"44":0.031296159260961874,"45":0.2717003992513425,"46":0.07899031632466799,"47":-0.008187185669595034,"48":0.13040554216434372,"49":-0.054422959064253866}},{"sx":1,"sy":1,"depth":50,"w":{"0":0.012539862237197925,"1":0.1986311903608974,"2":0.4185478176914634,"3":-0.23753138449602712,"4":-0.10462149169638747,"5":0.028954826107179324,"6":-0.048250667786281495,"7":-0.18707707252632866,"8":-0.14212538240080752,"9":-0.16159808811343954,"10":0.14184601978909012,"11":0.0733843517522459,"12":0.09013578159358847,"13":-0.0963866453281536,"14":0.17962071967797244,"15":-0.027705060875683262,"16":-0.06614460446631429,"17":-0.07892631438584756,"18":-0.11183748900013946,"19":0.21698127599478906,"20":-0.08871475040392779,"21":0.14203885464343513,"22":0.19759699906399839,"23":-0.02175408964323458,"24":0.12063975312867092,"25":-0.19407167545307571,"26":0.18357066049322862,"27":0.2017200963161157,"28":-0.060553269427030945,"29":-0.1048961146358563,"30":-0.13364916354986492,"31":-0.10000044882116418,"32":-0.3819825909405698,"33":0.1378092075980596,"34":0.10103475226717942,"35":0.13501512654412864,"36":-0.014284439989698174,"37":0.1861860482873279,"38":0.19238895608035314,"39":-0.17944933928411763,"40":-0.3605725955162564,"41":0.06357089051248613,"42":-0.07934041342524455,"43":0.0840834912736111,"44":0.22212727614256095,"45":0.21175912483464096,"46":0.11460998393767957,"47":-0.14706026132653843,"48":0.2606111546922114,"49":-0.19224449536649446}},{"sx":1,"sy":1,"depth":50,"w":{"0":0.010693630855895028,"1":-0.08850972308215545,"2":0.03277462525454513,"3":-0.06194199148062743,"4":0.0651194494156692,"5":-0.25299052764998825,"6":0.211703306267504,"7":-0.21980587607218183,"8":0.00007485226472331546,"9":0.08478391273251885,"10":0.4137317913139837,"11":0.2162680795263978,"12":-0.09426284858916432,"13":-0.09138452634206394,"14":-0.130148610504953,"15":-0.11955702073825968,"16":0.07592198648057362,"17":-0.3569271959391864,"18":-0.1401059215287983,"19":0.35533775849287297,"20":0.014383036182045734,"21":-0.09591873622004268,"22":0.004313129123034651,"23":-0.05360046081765838,"24":-0.0007629801029417349,"25":0.08633005647049474,"26":0.018105135118920354,"27":-0.030577036332640667,"28":0.12633191775439737,"29":0.1983537366153982,"30":0.2598161555337148,"31":-0.14019811677256666,"32":0.1552019485333174,"33":-0.01717046135779183,"34":0.029072138935388698,"35":-0.05935469046671948,"36":-0.1517042818120492,"37":-0.017677469846320855,"38":-0.06367007753562227,"39":-0.08457022157879775,"40":-0.13709872392181996,"41":0.14509937203643603,"42":0.08991877157695558,"43":0.00261144939348057,"44":-0.04206506626726432,"45":0.24702363383872705,"46":-0.11252277973674406,"47":0.03432340557899012,"48":0.01882104191044683,"49":-0.026523453687615058}}],"biases":{"sx":1,"sy":1,"depth":5,"w":{"0":0.002487068633708512,"1":0.012463665479524932,"2":-0.0005526539753702638,"3":-0.0027267808502766445,"4":-0.0026520435330062383}}},{"out_depth":5,"out_sx":1,"out_sy":1,"layer_type":"regression","num_inputs":5}]}

### Q4

Imagine you are training a computer program to solve the game Minesweeper with reinforcement learning. Describe what each of the following elements would be.

https://en.wikipedia.org/wiki/Minesweeper_(video_game)

https://minesweeper.online/

**Environment**

*your answer here*


**Agent**

*your answer here*

**Actions**

*your answer here*


**State**

*your answer here*


**Rewards**

*your answer here*